<a href="https://colab.research.google.com/github/karino2/US-patent-analysis/blob/triplet_loss_work/colab/bert_tripletloss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Colab porting of triplet loss trial for bert feature**

In [0]:
import os
import datetime
import pickle
import gzip
import pandas as pd
import numpy as np

import tensorflow as tf

tf.enable_eager_execution()

In [4]:
tf.__version__

'1.13.1'

In [5]:
!git clone -b docker https://github.com/yoheikikuta/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 220.79 KiB | 2.90 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [6]:
!ls

bert  sample_data


# Data setup

Feature data is created at bert-extract-feature.ipynb.

In [0]:
!mkdir ./bert/data

In [8]:
from google.colab import auth
auth.authenticate_user()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [9]:
!gsutil cp gs://karino2-uspatent/features/training_app_1000_features.pkl.gz bert/data/training_app_1000_features.pkl.gz
!gsutil cp gs://karino2-uspatent/features/test_app_1000_features.pkl.gz bert/data/test_app_1000_features.pkl.gz
!gsutil cp gs://karino2-uspatent/features/grants_2000_features.pkl.gz bert/data/grants_2000_features.pkl.gz

Copying gs://karino2-uspatent/features/training_app_1000_features.pkl.gz...
/ [1 files][  3.1 MiB/  3.1 MiB]                                                
Operation completed over 1 objects/3.1 MiB.                                      
Copying gs://karino2-uspatent/features/test_app_1000_features.pkl.gz...
/ [1 files][  3.1 MiB/  3.1 MiB]                                                
Operation completed over 1 objects/3.1 MiB.                                      
Copying gs://karino2-uspatent/features/grants_2000_features.pkl.gz...
/ [1 files][  7.9 MiB/  7.9 MiB]                                                
Operation completed over 1 objects/7.9 MiB.                                      


In [10]:
with gzip.open("./bert/data/training_app_1000_features.pkl.gz", 'rb') as f:
     train_features = pickle.load(f)
with gzip.open("./bert/data/test_app_1000_features.pkl.gz", 'rb') as f:
     test_features = pickle.load(f)
with gzip.open("./bert/data/grants_2000_features.pkl.gz", 'rb') as f:
     grants_features = pickle.load(f)
print(f"{train_features.shape}, {test_features.shape}, {grants_features.shape}")

(1000, 768), (1000, 768), (2524, 768)


In [11]:
!gsutil cp gs://karino2-uspatent/citations_info_2000.df.gz ./bert/data/
!gsutil cp gs://karino2-uspatent/testset_app_1000.df.gz ./bert/data/
!gsutil cp gs://karino2-uspatent/training_app_1000.df.gz ./bert/data/
!gsutil cp gs://karino2-uspatent/grants_for_2000.df.gz ./bert/data/

Copying gs://karino2-uspatent/citations_info_2000.df.gz...
/ [1 files][234.3 KiB/234.3 KiB]                                                
Operation completed over 1 objects/234.3 KiB.                                    
Copying gs://karino2-uspatent/testset_app_1000.df.gz...
- [1 files][ 13.4 MiB/ 13.4 MiB]                                                
Operation completed over 1 objects/13.4 MiB.                                     
Copying gs://karino2-uspatent/training_app_1000.df.gz...
/ [1 files][ 14.2 MiB/ 14.2 MiB]                                                
Operation completed over 1 objects/14.2 MiB.                                     
Copying gs://karino2-uspatent/grants_for_2000.df.gz...
-
Operation completed over 1 objects/44.6 MiB.                                     


In [0]:
citations_info_target = pd.read_pickle("./bert/data/citations_info_2000.df.gz")
train_app_df = pd.read_pickle("./bert/data/training_app_1000.df.gz")
test_app_df = pd.read_pickle("./bert/data/testset_app_1000.df.gz")
grants_target_df = pd.read_pickle("./bert/data/grants_for_2000.df.gz")

### Prepare data for circulum tripletloss learning

In [0]:
import random
import pandas as pd
import numpy as np
import pickle
from collections import OrderedDict

In [0]:
train_normalized_feature_dict_1000 = { 
    k:v/np.linalg.norm(v) for k,v in zip(train_app_df['app_id'], train_features[:, ])
}
grants_normalized_feature_dict_2000 = { 
    k:v/np.linalg.norm(v) for k,v in zip(grants_target_df['parsed'], grants_features[:, ])
}

Confirm result

In [19]:
train_app_df.head()

,app_id,xml
0,14222691,"<us-patent-application lang=""EN"" dtd-version=""..."
1,12515852,"<us-patent-application lang=""EN"" dtd-version=""..."
2,12033424,"<us-patent-application lang=""EN"" dtd-version=""..."
3,12402344,"<us-patent-application lang=""EN"" dtd-version=""..."
4,12155425,"<us-patent-application lang=""EN"" dtd-version=""..."


In [25]:
train_normalized_feature_dict_1000[14222691][0:5]

array([-0.05969358,  0.05039638, -0.0200557 ,  0.04519269, -0.01494433])

In [26]:
train_features[0, 0:5]

array([-1.04389739,  0.88131171, -0.35072595,  0.79031169, -0.26134047])

In [28]:
(train_features[0, ]/np.linalg.norm(train_features[0, ]))[0:5]

array([-0.05969358,  0.05039638, -0.0200557 ,  0.04519269, -0.01494433])

### Calculate cosin similarity for whole pair

In [15]:
%%time

sim_dict = {
    app_id:{ parsed:np.sum(train_normalized_feature_dict_1000[app_id]*grants_normalized_feature_dict_2000[parsed])
            for parsed 
            in grants_target_df['parsed'] } 
    for app_id 
    in train_app_df['app_id']
}

CPU times: user 13.7 s, sys: 181 ms, total: 13.8 s
Wall time: 13.8 s


check result

In [32]:
list(sim_dict.keys())[0]

14222691

In [46]:
sorted( sim_dict[12411468].values(), reverse=True )[:5]

[0.8536353801810572,
 0.8500573288849081,
 0.8478197928831097,
 0.8464742417020769,
 0.8457543474536907]

In [36]:
citations_info_target[citations_info_target.app_id == 12411468].parsed

4089    7061154
4090    7061154
Name: parsed, dtype: int64

In [44]:
grants_target_df[grants_target_df.parsed == 7061154].index[0]

540

In [45]:
list(sim_dict[12411468].values())[540]

0.8435186037581648

In [47]:
sorted( sim_dict[12411468].values(), reverse=True ).index(0.8435186037581648)

7

### Create triplet

In [0]:
def sort_similarity_by_value(sim_dict, app_id):
    '''
    input:
        sim_dict: similary dictionary
        app_id: target application id
    return:
        [(parsed1, sim1), (parsed2, sim2), ...] sorted by similarities
    '''
    return [(parsed, sim_dict[app_id][parsed]) for parsed in sorted(sim_dict[app_id], key=sim_dict[app_id].get)]
  
  
def get_cited_grants(citations_info_target, app_id):
    '''
    input:
        citations_info_target: DataFrame of citation relationships
        app_id: target application id
    return:
        {parsed1, parsed2, ...} that are cited to reject app_id
    '''
    return set(citations_info_target[citations_info_target['app_id'] == app_id]['parsed'])
  
def make_uncited_grants_for_app_id(sim_dict, citations_info_target, app_id, sidx, eidx, num, shuffle=True):
    '''
    input:
        sim_dict: 
        citations_info_target:
        app_id: target application id
        sidx: start index to slice the sorted (parsed, sim) list
        eidx: end index to slice the sorted (parsed, sim) list
        num: number of grants that will be returned
    return:
        [parsed_1, parsed_2, ..., parsed_num] that are NOT cited to reject app_id
    '''
    sorted_grants_list = sort_similarity_by_value(sim_dict, app_id)
    sorted_grants_list = sorted_grants_list[sidx:eidx]
    if shuffle:
        random.shuffle(sorted_grants_list)
    
    cited_grants = get_cited_grants(citations_info_target, app_id)
    uncited_grants = []
    
    idx = 0
    while len(uncited_grants) != num:
        grant_id, _ = sorted_grants_list[idx]
        if not grant_id in cited_grants:
            uncited_grants.append(grant_id)
        idx += 1
    
    return uncited_grants  

In [11]:
random.seed(0)
make_uncited_grants_for_app_id(sim_dict, citations_info_target, 12411468, 0, 100, 4)

[7196326, 6979916, 6971650, 6899075]

In [50]:
random.seed(1)
make_uncited_grants_for_app_id(sim_dict, citations_info_target, 12411468, 0, 100, 4)

[7108673, 7846876, 7168415, 7052189]

In [0]:
NEGATIVE_SAMPLE_RATIO=1

In [0]:
def create_triplet_pairs(sidx, eidx):
    all_elems = []
    
    for app_id in train_app_df['app_id']:
        cited_grants = get_cited_grants(citations_info_target, app_id)
        num_cited_grants = len(cited_grants)
        uncited_grants = make_uncited_grants_for_app_id(
            sim_dict, citations_info_target, app_id, sidx, eidx, num_cited_grants*NEGATIVE_SAMPLE_RATIO)
        
        for idx, cited in enumerate(cited_grants):
          for grants_idx in range(NEGATIVE_SAMPLE_RATIO):
            all_elems.append([app_id, cited, uncited_grants[idx*NEGATIVE_SAMPLE_RATIO+grants_idx]])
    
    result_df = pd.DataFrame(all_elems)
    result_df.columns = ['app_id', 'cited_grants', 'uncited_grants']
    
    return result_df

In [0]:
random.seed(0)
NEGATIVE_SAMPLE_RATIO=4
test = create_triplet_pairs(0, 100)

In [60]:
test.head(10)

,app_id,cited_grants,uncited_grants
0,14222691,8206188,7474966
1,14222691,8206188,6925385
2,14222691,8206188,7735351
3,14222691,8206188,7750029
4,14222691,8177561,7797096
5,14222691,8177561,7879828
6,14222691,8177561,7856913
7,14222691,8177561,7605249
8,14222691,8179692,7656129
9,14222691,8179692,7781650


In [0]:
NEGATIVE_SAMPLE_RATIO=1

In [55]:
len(test)

1282

In [0]:
random.seed(1)
test = create_triplet_pairs(0, 100)

In [57]:
test.head(2)

,app_id,cited_grants,uncited_grants
0,14222691,8206188,6977086
1,14222691,8177561,7252786


### Train Model

In [20]:
tf.executing_eagerly() 

True

In [0]:
class Model(object):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.W = tf.Variable( tf.random_normal( [self.input_shape, self.output_shape] ), name='weight' )
        self.B = tf.Variable( tf.random_normal( [self.output_shape] ), name='bias' ) 
        self.variables = [ self.W, self.B ]
    
    def frwrd_pass(self,X_train):
        out = tf.matmul( X_train, self.W ) + self.B
        
        return out

In [0]:
def tripletloss(anchor_out, positive_out, negative_out, margin=0.2):
    norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
    norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
    norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)
    
    d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
    d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)
    
    loss = tf.maximum(0.0, margin + d_pos - d_neg)
    
    return tf.reduce_mean(loss)

In [0]:
def create_training_input_np(sidx, eidx):
    anchor_list = []
    positive_list = []
    negative_list = []
    
    triplet_pairs = create_triplet_pairs(sidx, eidx)
    
    for row in triplet_pairs.itertuples():
        anchor_list.append(train_normalized_feature_dict_1000[row.app_id])
        positive_list.append(grants_normalized_feature_dict_2000[row.cited_grants])
        negative_list.append(grants_normalized_feature_dict_2000[row.uncited_grants])
    
    return np.array([np.array(anchor_list), np.array(positive_list), np.array(negative_list)])

In [0]:
def calc_loss_with_changing_negative_pair(sidx, eidx, lossfun=tripletloss, batch_size=10):
  print(calc_loss_with_changing_negative_pair_without_print(sidx, eidx, lossfun, batch_size))

def calc_loss_with_changing_negative_pair_without_print(sidx, eidx, lossfun=tripletloss, batch_size=10):
    seed = 0
    all_loss = []
    random.seed(seed)

    input_data_np = create_training_input_np(sidx, eidx)
    data_num = int(input_data_np.shape[1])
    rand_idx = np.random.permutation(data_num)
    index_data_np = np.array([
        input_data_np[0][rand_idx], 
        input_data_np[1][rand_idx], 
        input_data_np[2][rand_idx]])

    input_data = tf.convert_to_tensor(input_data_np, dtype=tf.float32)
    anchor_data, positive_data, negative_data = input_data

    for iter_id in range(data_num // batch_size):
      anchor_out = model.frwrd_pass(anchor_data[iter_id*batch_size : (iter_id+1)*batch_size])
      positive_out = model.frwrd_pass(positive_data[iter_id*batch_size : (iter_id+1)*batch_size])
      negative_out = model.frwrd_pass(negative_data[iter_id*batch_size : (iter_id+1)*batch_size])
      curr_loss = lossfun(anchor_out, positive_out, negative_out)
      all_loss.append(curr_loss)

    return (sum(all_loss)/len(all_loss))

In [241]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.01658673, shape=(), dtype=float32)
tf.Tensor(0.09671563, shape=(), dtype=float32)
tf.Tensor(0.1443036, shape=(), dtype=float32)


In [0]:
def train_with_changing_negative_pair_with_loss(sidx, eidx, lossfun, batch_size, epochs):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.00001)
    
    seed = 0
    for i in range(epochs):
        seed += 1
        random.seed(seed)
        
        input_data_np = create_training_input_np(sidx, eidx)
        data_num = int(input_data_np.shape[1])
        rand_idx = np.random.permutation(data_num)
        index_data_np = np.array([
            input_data_np[0][rand_idx], 
            input_data_np[1][rand_idx], 
            input_data_np[2][rand_idx]])

        input_data = tf.convert_to_tensor(input_data_np, dtype=tf.float32)
        anchor_data, positive_data, negative_data = input_data

        for iter_id in range(data_num // batch_size):        
            with tf.GradientTape() as tape:
                anchor_out = model.frwrd_pass(anchor_data[iter_id*batch_size : (iter_id+1)*batch_size])
                positive_out = model.frwrd_pass(positive_data[iter_id*batch_size : (iter_id+1)*batch_size])
                negative_out = model.frwrd_pass(negative_data[iter_id*batch_size : (iter_id+1)*batch_size])
                curr_loss = lossfun(anchor_out, positive_out, negative_out)
            grads = tape.gradient( curr_loss, model.variables )
            optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())

        if i % 10 == 0:
            print( "Loss at step {:d}: {:.5f}".format(i, curr_loss) )

def train_with_changing_negative_pair(sidx, eidx, batch_size, epochs):
  train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size, epochs)

In [0]:
# start_end_index_pairs = (
#     (0, 100),
#     (100, 200),
#     (200, 300),
#     (300, 400),
#     (400, 500),
#     (500, 600),
#     (600, 700),
#     (700, 800),
#     (800, 900),
#     (900, 1000),
#     (1000, 1100),
#     (1100, 1200),
#     (1200, 1300),
#     (1300, 1400),
#     (1400, 1500),
#     (1500, 1600),
#     (1600, 1700),
#     (1700, 1800),
#     (1800, 1900),
#     (1900, 2000)
# )

# start_end_index_pairs = (
#     (0, 400),
#     (400, 800),
#     (800, 1200),
#     (1200, 1600),
#     (1600, 2000)
# )

start_end_index_pairs = (
    (0, 100),
    (900, 1000),
    (1900, 2000)
)

In [0]:
model = Model(input_shape=768, output_shape=100)

In [242]:
%%time

# for sidx, eidx in start_end_index_pairs:
#     print("   start index: {}, end index: {}".format(sidx,eidx))
#     train_with_changing_negative_pair(sidx, eidx, batch_size=10, epochs=31)

# for sidx, eidx in start_end_index_pairs:
#     print("   start index: {}, end index: {}".format(sidx,eidx))
#     train_with_changing_negative_pair(sidx, eidx, batch_size=10, epochs=51)

for sidx, eidx in start_end_index_pairs:
    print("   start index: {}, end index: {}".format(sidx,eidx))
    train_with_changing_negative_pair(sidx, eidx, batch_size=10, epochs=21)

   start index: 0, end index: 100
Loss at step 0: 0.02616
Loss at step 10: 0.00717
Loss at step 20: 0.01298
   start index: 900, end index: 1000
Loss at step 0: 0.09295
Loss at step 10: 0.09062
Loss at step 20: 0.06588
   start index: 1900, end index: 2000
Loss at step 0: 0.09322
Loss at step 10: 0.09928
Loss at step 20: 0.06732
CPU times: user 4min 46s, sys: 4.02 s, total: 4min 50s
Wall time: 4min 34s


In [243]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.00667771, shape=(), dtype=float32)
tf.Tensor(0.027310502, shape=(), dtype=float32)
tf.Tensor(0.072016366, shape=(), dtype=float32)


In [23]:
# I do not save model these days.

# os.makedirs('../trained_model/tripletloss_circulum', exist_ok=True)
# saver = tf.contrib.eager.Saver(model.variables)
# saver.save("../trained_model/tripletloss_circulum/ckpt")

'../trained_model/tripletloss_circulum/ckpt'

### Inferece with trained model

Saver seems fragile in eager mode. I use trained model on memory.

In [0]:
test_normalized_feature_dict_1000 = { 
    k:v/np.linalg.norm(v) for k,v in zip(test_app_df['app_id'], test_features[:, ])
}

In [0]:
def to_extracted_features(normalized_feature_dict, model):
  # doc_id is app_id for application, pared for grants.
  sorted_keys = sorted(normalized_feature_dict.keys())

  input_tensors = tf.convert_to_tensor(
      np.array([ normalized_feature_dict[k] for k in sorted_keys ]),
      dtype=tf.float32)
  extracted_features = model.frwrd_pass(input_tensors).numpy()
  return pd.DataFrame({ 
    'doc_id':sorted_keys, 'extracted_feature':[ v/np.linalg.norm(v) for v in extracted_features ]})

In [0]:
import collections

def calc_all_ranks(all_app_ids, sim_dic):
  all_ranks = []

  for app_id in all_app_ids:
      cited_grants = get_cited_grants(citations_info_target, app_id)
      sorted_kv = reversed(sort_similarity_by_value(sim_dic, app_id))  # higher score, similar patent

      idx = 1
      for k,v in sorted_kv:
          if k in cited_grants:
              all_ranks.append(idx)
          idx += 1
  return all_ranks

def calc_all_ranks_counter(all_app_ids, sim_dic):
  return collections.Counter(calc_all_ranks(all_app_ids, sim_dic))

In [0]:
test_extracted_features_df = to_extracted_features(test_normalized_feature_dict_1000, model)
grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, model)

In [245]:
test_extracted_features_df.head(2)

,doc_id,extracted_feature
0,12000862,"[-0.06014522, 0.15319835, -0.094469905, 0.0928..."
1,12003258,"[-0.14375061, 0.054770324, 0.052622166, 0.2140..."


In [246]:
grants_extracted_features_df.head(2)

,doc_id,extracted_feature
0,6837383,"[0.17937347, 0.06213054, -0.0074147717, 0.2118..."
1,6837647,"[0.15643527, 0.080645256, -0.041111432, 0.1119..."


In [0]:
def create_sim_dict(test_extracted_df, grants_extracted_df):
  return {
    app_id:{ parsed:np.sum(test_f*grants_f)
            for parsed, grants_f 
            in zip(grants_extracted_df['doc_id'], grants_extracted_df['extracted_feature']) } 
    for app_id, test_f 
    in zip(test_extracted_df['doc_id'], test_extracted_df['extracted_feature'])
  }

In [247]:
%%time

sim_dict_by_extracted = create_sim_dict(test_extracted_features_df, grants_extracted_features_df)

CPU times: user 10.1 s, sys: 153 ms, total: 10.3 s
Wall time: 10.3 s


In [250]:
counter = calc_all_ranks_counter(test_extracted_features_df['doc_id'], sim_dict_by_extracted)
print(counter)

Counter({1: 131, 2: 61, 3: 44, 7: 27, 6: 27, 4: 26, 5: 25, 11: 19, 10: 16, 15: 15, 22: 15, 9: 13, 19: 13, 34: 12, 25: 11, 31: 11, 18: 11, 21: 11, 12: 10, 8: 10, 24: 10, 23: 10, 17: 9, 27: 9, 13: 9, 14: 8, 26: 7, 55: 7, 43: 7, 41: 7, 38: 7, 30: 7, 36: 6, 16: 6, 69: 5, 28: 5, 37: 5, 62: 5, 49: 5, 54: 5, 182: 5, 134: 5, 50: 5, 51: 5, 130: 5, 84: 5, 33: 5, 48: 4, 186: 4, 52: 4, 35: 4, 127: 4, 91: 4, 32: 4, 20: 4, 65: 4, 68: 4, 46: 4, 120: 4, 449: 4, 101: 4, 56: 4, 57: 3, 286: 3, 61: 3, 63: 3, 106: 3, 128: 3, 53: 3, 129: 3, 125: 3, 206: 3, 76: 3, 108: 3, 167: 3, 39: 3, 107: 3, 190: 3, 73: 3, 168: 3, 191: 3, 117: 3, 110: 3, 132: 3, 44: 3, 72: 3, 243: 3, 149: 3, 40: 3, 112: 3, 196: 3, 171: 3, 159: 3, 29: 3, 151: 3, 672: 2, 60: 2, 161: 2, 266: 2, 118: 2, 96: 2, 162: 2, 443: 2, 166: 2, 193: 2, 201: 2, 1002: 2, 165: 2, 42: 2, 75: 2, 126: 2, 119: 2, 722: 2, 185: 2, 89: 2, 501: 2, 306: 2, 331: 2, 184: 2, 183: 2, 321: 2, 622: 2, 239: 2, 359: 2, 311: 2, 59: 2, 133: 2, 64: 2, 93: 2, 99: 2, 312: 2, 77

In [251]:
# counter is started from 1.

sum(counter[i] for i in range(11))

380

### Use raw feature as is (for comparison)

In [47]:
%%time

sim_dict = {
    app_id:{ parsed:np.sum(test_normalized_feature_dict_1000[app_id]*grants_normalized_feature_dict_2000[parsed])
            for parsed 
            in grants_target_df['parsed'] } 
    for app_id 
    in test_app_df['app_id']
}

CPU times: user 13.3 s, sys: 149 ms, total: 13.4 s
Wall time: 13.4 s


In [48]:
%%time

all_ranks = []

for app_id in test_app_df['app_id']:
    cited_grants = get_cited_grants(citations_info_target, app_id)
    sorted_kv = reversed(sort_similarity_by_value(sim_dict, app_id))  # higher score, similar patent

    idx = 1
    for k,v in sorted_kv:
        if k in cited_grants:
            all_ranks.append(idx)
        idx += 1

CPU times: user 2.89 s, sys: 8.73 ms, total: 2.9 s
Wall time: 2.89 s


In [49]:
counter = collections.Counter(all_ranks)
print(counter)

Counter({1: 178, 2: 73, 3: 46, 5: 40, 4: 38, 6: 31, 7: 19, 9: 17, 8: 16, 10: 15, 13: 13, 14: 13, 11: 13, 15: 12, 18: 12, 16: 11, 12: 10, 26: 9, 19: 9, 34: 9, 21: 8, 23: 8, 38: 7, 28: 7, 50: 6, 39: 6, 37: 6, 33: 6, 53: 6, 100: 5, 47: 5, 76: 5, 35: 5, 64: 5, 29: 5, 73: 5, 61: 5, 32: 5, 17: 5, 68: 5, 31: 5, 52: 4, 59: 4, 40: 4, 143: 4, 90: 4, 48: 4, 63: 4, 44: 4, 49: 4, 20: 4, 65: 4, 70: 4, 30: 4, 83: 3, 168: 3, 147: 3, 114: 3, 22: 3, 444: 3, 93: 3, 43: 3, 121: 3, 122: 3, 189: 3, 148: 3, 51: 3, 57: 3, 46: 3, 25: 3, 385: 3, 66: 3, 85: 3, 125: 3, 304: 3, 36: 3, 195: 3, 92: 3, 187: 3, 69: 3, 99: 3, 411: 2, 24: 2, 131: 2, 262: 2, 724: 2, 72: 2, 454: 2, 396: 2, 87: 2, 130: 2, 413: 2, 89: 2, 157: 2, 292: 2, 1182: 2, 264: 2, 133: 2, 78: 2, 174: 2, 56: 2, 67: 2, 227: 2, 134: 2, 163: 2, 97: 2, 126: 2, 517: 2, 213: 2, 128: 2, 110: 2, 112: 2, 173: 2, 62: 2, 42: 2, 179: 2, 378: 2, 155: 2, 88: 2, 376: 2, 505: 2, 183: 2, 167: 2, 105: 2, 45: 2, 209: 2, 211: 2, 74: 2, 75: 2, 415: 2, 617: 2, 169: 2, 615: 

In [50]:
sum(counter[i] for i in range(11))

473

# Changing loss or model

### Triplet loss, positive doubled

In [0]:
def tipletloss_pos_ntimes(anchor_out, positive_out, negative_out, ntimes=2, margin=0.2):
    norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
    norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
    norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)
    
    d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
    d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)
    
    loss = tf.maximum(0.0, margin + 2*d_pos - d_neg)
    
    return tf.reduce_mean(loss)
  
def tripletloss_posdouble(anchor_out, positive_out, negative_out, margin=0.2):
  return tipletloss_pos_ntimes(anchor_out, positive_out, negative_out, 2, margin)

def tripletloss_pos_10times(anchor_out, positive_out, negative_out, margin=0.2):
  return tipletloss_pos_ntimes(anchor_out, positive_out, negative_out, 2, margin)



In [0]:
model = Model(input_shape=768, output_shape=100)

In [215]:
%%time

for sidx, eidx in start_end_index_pairs:
    print("   start index: {}, end index: {}".format(sidx,eidx))
    train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss_posdouble, batch_size=10, epochs=21)

   start index: 0, end index: 100
Loss at step 0: 0.14810
Loss at step 10: 0.12633
Loss at step 20: 0.10907
   start index: 900, end index: 1000
Loss at step 0: 0.25632
Loss at step 10: 0.24276
Loss at step 20: 0.24377
   start index: 1900, end index: 2000
Loss at step 0: 0.26960
Loss at step 10: 0.26411
Loss at step 20: 0.26362
CPU times: user 4min 50s, sys: 4.33 s, total: 4min 55s
Wall time: 4min 39s


In [0]:
test_extracted_features_df = to_extracted_features(test_normalized_feature_dict_1000, model)

In [219]:
test_extracted_features_df.head()

,doc_id,extracted_feature
0,12000862,"[-0.013157393, 0.06550002, -0.17330328, 0.0534..."
1,12003258,"[-0.01271724, -0.021055741, -0.18226379, 0.150..."
2,12004701,"[-0.059054796, -0.022013474, -0.1141999, 0.041..."
3,12007341,"[-0.0034107184, -0.013823663, -0.15175237, 0.0..."
4,12018798,"[-0.017291823, -0.04676495, -0.09946125, 0.064..."


In [0]:
grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, model)

In [221]:
grants_extracted_features_df.head()

,doc_id,extracted_feature
0,6837383,"[0.01180602, -0.023277633, -0.059893485, 0.048..."
1,6837647,"[0.014063301, 0.02104149, -0.06873307, 0.01846..."
2,6837799,"[0.020941963, -0.0022532253, -0.12189834, 0.00..."
3,6837893,"[0.044268582, 0.018448185, -0.06050285, 0.0270..."
4,6837910,"[0.015180708, 0.11016171, -0.12653644, 0.11364..."


In [223]:
%%time
doubled_sim_dict = create_sim_dict(test_extracted_features_df, grants_extracted_features_df)

CPU times: user 9.99 s, sys: 77.3 ms, total: 10.1 s
Wall time: 10.1 s


In [227]:
counter = calc_all_ranks_counter(test_extracted_features_df['doc_id'], doubled_sim_dict)
print(counter)

Counter({1: 150, 2: 55, 3: 39, 4: 29, 6: 28, 5: 27, 8: 19, 11: 17, 7: 16, 13: 16, 17: 16, 10: 16, 9: 13, 12: 12, 16: 11, 23: 11, 27: 11, 21: 10, 15: 10, 19: 9, 33: 9, 20: 8, 14: 8, 22: 8, 32: 8, 24: 7, 31: 6, 41: 6, 48: 6, 65: 6, 38: 6, 44: 6, 35: 5, 68: 5, 28: 5, 25: 5, 153: 5, 62: 5, 43: 5, 55: 5, 34: 5, 105: 5, 148: 4, 92: 4, 51: 4, 164: 4, 78: 4, 46: 4, 87: 4, 26: 4, 56: 4, 67: 4, 350: 4, 192: 4, 57: 4, 120: 4, 61: 4, 145: 3, 128: 3, 155: 3, 45: 3, 144: 3, 18: 3, 85: 3, 121: 3, 86: 3, 95: 3, 137: 3, 49: 3, 29: 3, 54: 3, 53: 3, 83: 3, 230: 3, 64: 3, 106: 3, 113: 3, 579: 3, 93: 3, 47: 3, 97: 3, 123: 3, 77: 3, 42: 3, 40: 3, 58: 3, 50: 3, 147: 3, 329: 2, 456: 2, 90: 2, 150: 2, 401: 2, 260: 2, 1248: 2, 177: 2, 213: 2, 376: 2, 94: 2, 89: 2, 245: 2, 100: 2, 116: 2, 167: 2, 96: 2, 1140: 2, 66: 2, 216: 2, 232: 2, 426: 2, 172: 2, 251: 2, 139: 2, 347: 2, 156: 2, 72: 2, 366: 2, 195: 2, 122: 2, 539: 2, 169: 2, 574: 2, 288: 2, 88: 2, 509: 2, 80: 2, 52: 2, 749: 2, 173: 2, 103: 2, 1489: 2, 186: 2,

Positive 10 times.

In [0]:
model = Model(input_shape=768, output_shape=100)

In [231]:
%%time

for sidx, eidx in start_end_index_pairs:
    print("   start index: {}, end index: {}".format(sidx,eidx))
    train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss_pos_10times, batch_size=10, epochs=21)

   start index: 0, end index: 100
Loss at step 0: 0.17560
Loss at step 10: 0.15177
Loss at step 20: 0.12895
   start index: 900, end index: 1000
Loss at step 0: 0.25382
Loss at step 10: 0.24570
Loss at step 20: 0.23713
   start index: 1900, end index: 2000
Loss at step 0: 0.26674
Loss at step 10: 0.26804
Loss at step 20: 0.26128
CPU times: user 4min 45s, sys: 4.21 s, total: 4min 49s
Wall time: 4min 33s


In [0]:
test_extracted_features_df = to_extracted_features(test_normalized_feature_dict_1000, model)
grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, model)
ten_sim_dict = create_sim_dict(test_extracted_features_df, grants_extracted_features_df)

In [233]:
counter = calc_all_ranks_counter(test_extracted_features_df['doc_id'], ten_sim_dict)
print(counter)

Counter({1: 137, 2: 60, 3: 42, 4: 42, 7: 26, 5: 23, 6: 23, 9: 20, 8: 15, 11: 15, 10: 14, 23: 13, 15: 12, 18: 11, 14: 11, 12: 9, 59: 9, 20: 8, 55: 8, 26: 8, 16: 8, 19: 7, 49: 7, 38: 7, 17: 7, 22: 7, 35: 7, 56: 7, 43: 7, 32: 7, 25: 7, 13: 6, 77: 6, 24: 6, 34: 6, 30: 6, 48: 6, 39: 6, 61: 6, 42: 5, 21: 5, 40: 5, 58: 5, 27: 5, 69: 5, 72: 5, 29: 5, 80: 5, 36: 5, 143: 4, 146: 4, 75: 4, 33: 4, 65: 4, 54: 4, 71: 4, 50: 4, 37: 4, 46: 4, 31: 4, 63: 4, 92: 3, 68: 3, 103: 3, 94: 3, 145: 3, 60: 3, 82: 3, 170: 3, 109: 3, 123: 3, 62: 3, 227: 3, 126: 3, 90: 3, 99: 3, 288: 3, 147: 3, 52: 3, 98: 3, 28: 3, 174: 3, 249: 3, 137: 3, 47: 3, 121: 2, 86: 2, 155: 2, 87: 2, 144: 2, 51: 2, 177: 2, 599: 2, 141: 2, 1199: 2, 214: 2, 101: 2, 135: 2, 256: 2, 344: 2, 108: 2, 136: 2, 436: 2, 79: 2, 70: 2, 226: 2, 930: 2, 89: 2, 743: 2, 53: 2, 201: 2, 215: 2, 2400: 2, 618: 2, 298: 2, 225: 2, 166: 2, 128: 2, 468: 2, 57: 2, 186: 2, 102: 2, 326: 2, 66: 2, 1224: 2, 267: 2, 369: 2, 78: 2, 786: 2, 100: 2, 152: 2, 591: 2, 164: 2

Training is not enough?

In [234]:
sidx, eidx = 1900, 2000
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss_pos_10times, batch_size=10, epochs=51)

   start index: 1900, end index: 2000
Loss at step 0: 0.25474
Loss at step 10: 0.25733
Loss at step 20: 0.25183
Loss at step 30: 0.24196
Loss at step 40: 0.23452
Loss at step 50: 0.23559


In [0]:
test_extracted_features_df = to_extracted_features(test_normalized_feature_dict_1000, model)
grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, model)
ten_sim_dict = create_sim_dict(test_extracted_features_df, grants_extracted_features_df)

In [236]:
counter = calc_all_ranks_counter(test_extracted_features_df['doc_id'], ten_sim_dict)
print(counter)

Counter({1: 145, 2: 64, 3: 52, 4: 34, 5: 28, 6: 25, 9: 24, 11: 22, 7: 19, 12: 17, 8: 16, 25: 14, 10: 12, 22: 10, 18: 10, 28: 10, 16: 10, 19: 9, 26: 9, 14: 9, 43: 9, 33: 9, 61: 7, 15: 7, 35: 7, 56: 7, 41: 7, 24: 7, 17: 7, 48: 7, 20: 7, 65: 6, 42: 6, 32: 6, 57: 6, 39: 6, 54: 6, 180: 6, 31: 6, 13: 6, 60: 6, 29: 6, 58: 5, 49: 5, 102: 5, 37: 5, 105: 5, 191: 5, 88: 4, 69: 4, 27: 4, 79: 4, 23: 4, 40: 4, 52: 4, 86: 4, 38: 4, 76: 4, 62: 4, 113: 4, 629: 3, 218: 3, 270: 3, 21: 3, 87: 3, 148: 3, 64: 3, 562: 3, 45: 3, 141: 3, 116: 3, 46: 3, 85: 3, 78: 3, 92: 3, 89: 3, 172: 3, 77: 3, 104: 3, 106: 3, 47: 3, 59: 3, 167: 3, 96: 3, 185: 3, 55: 3, 66: 3, 833: 2, 97: 2, 698: 2, 238: 2, 132: 2, 94: 2, 513: 2, 186: 2, 99: 2, 51: 2, 188: 2, 75: 2, 158: 2, 154: 2, 201: 2, 209: 2, 1372: 2, 208: 2, 91: 2, 125: 2, 63: 2, 112: 2, 293: 2, 567: 2, 335: 2, 1201: 2, 160: 2, 482: 2, 407: 2, 928: 2, 451: 2, 262: 2, 768: 2, 164: 2, 385: 2, 139: 2, 354: 2, 153: 2, 448: 2, 53: 2, 312: 2, 230: 2, 267: 2, 72: 2, 126: 2, 138

### More complex model

In [0]:
class ComplexModel(object):
    def __init__(self, input_shape, output_shape1, output_shape2):
        self.input_shape = input_shape
        self.output_shape1 = output_shape1
        self.output_shape2 = output_shape2
        self.W1 = tf.Variable( tf.random_normal( [self.input_shape, self.output_shape1] ), name='weight' )
        self.B1 = tf.Variable( tf.random_normal( [self.output_shape1] ), name='bias' ) 
        self.W2 = tf.Variable( tf.random_normal( [self.output_shape1, self.output_shape2] ), name='weight' )
        self.B2 = tf.Variable( tf.random_normal( [self.output_shape2] ), name='bias' ) 
        self.variables = [ self.W1, self.B1, self.W2, self.B2 ]

    def frwrd_pass(self,X_train):
        out = tf.matmul( X_train, self.W1 ) + self.B1
        out = tf.nn.relu(out)
        out = tf.matmul( out, self.W2 ) + self.B2

        return out

In [0]:
model = ComplexModel(input_shape=768, output_shape1=768, output_shape2=100)

In [255]:
calc_loss_with_changing_negative_pair(0, 100, tripletloss_posdouble)
calc_loss_with_changing_negative_pair(900, 1000, tripletloss_posdouble)
calc_loss_with_changing_negative_pair(1900, 2000, tripletloss_posdouble)

tf.Tensor(0.15582651, shape=(), dtype=float32)
tf.Tensor(0.21938041, shape=(), dtype=float32)
tf.Tensor(0.24572179, shape=(), dtype=float32)


In [256]:
%%time

for sidx, eidx in start_end_index_pairs:
    print("   start index: {}, end index: {}".format(sidx,eidx))
    train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss_posdouble, batch_size=10, epochs=21)

   start index: 0, end index: 100
Loss at step 0: 0.16603
Loss at step 10: 0.12623
Loss at step 20: 0.09673
   start index: 900, end index: 1000
Loss at step 0: 0.24931
Loss at step 10: 0.23911
Loss at step 20: 0.19546
   start index: 1900, end index: 2000
Loss at step 0: 0.27353
Loss at step 10: 0.25290
Loss at step 20: 0.22856
CPU times: user 7min 15s, sys: 9.77 s, total: 7min 25s
Wall time: 6min 12s


In [257]:
calc_loss_with_changing_negative_pair(0, 100, tripletloss_posdouble)
calc_loss_with_changing_negative_pair(900, 1000, tripletloss_posdouble)
calc_loss_with_changing_negative_pair(1900, 2000, tripletloss_posdouble)

tf.Tensor(0.09658408, shape=(), dtype=float32)
tf.Tensor(0.16320944, shape=(), dtype=float32)
tf.Tensor(0.21651158, shape=(), dtype=float32)


In [0]:
test_extracted_features_df = to_extracted_features(test_normalized_feature_dict_1000, model)
grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, model)
complex_sim_dict = create_sim_dict(test_extracted_features_df, grants_extracted_features_df)

In [259]:
counter = calc_all_ranks_counter(test_extracted_features_df['doc_id'], complex_sim_dict)
print(counter)

Counter({1: 124, 2: 58, 3: 41, 6: 28, 4: 28, 5: 27, 8: 25, 7: 20, 18: 15, 14: 14, 9: 13, 10: 12, 13: 12, 21: 11, 12: 10, 17: 10, 35: 10, 42: 9, 11: 9, 15: 9, 40: 9, 19: 9, 28: 9, 27: 9, 53: 8, 32: 8, 20: 8, 49: 7, 29: 7, 41: 7, 37: 7, 16: 7, 25: 7, 43: 7, 26: 6, 50: 6, 58: 6, 24: 6, 73: 6, 47: 6, 142: 6, 34: 5, 75: 5, 22: 5, 23: 5, 135: 5, 33: 5, 111: 5, 92: 5, 107: 5, 38: 5, 44: 5, 110: 4, 60: 4, 138: 4, 74: 4, 57: 4, 126: 4, 62: 4, 103: 4, 31: 4, 56: 4, 91: 4, 175: 4, 221: 4, 67: 4, 127: 4, 72: 4, 61: 4, 100: 3, 30: 3, 39: 3, 106: 3, 183: 3, 55: 3, 81: 3, 125: 3, 166: 3, 120: 3, 117: 3, 84: 3, 136: 3, 102: 3, 69: 3, 139: 3, 59: 3, 70: 3, 63: 3, 104: 3, 325: 3, 361: 3, 485: 3, 370: 3, 130: 3, 98: 3, 66: 3, 77: 3, 244: 3, 425: 3, 95: 3, 105: 3, 205: 3, 82: 3, 508: 3, 71: 3, 113: 2, 64: 2, 1851: 2, 354: 2, 184: 2, 240: 2, 87: 2, 268: 2, 89: 2, 172: 2, 218: 2, 115: 2, 122: 2, 83: 2, 118: 2, 153: 2, 51: 2, 358: 2, 146: 2, 133: 2, 754: 2, 333: 2, 152: 2, 79: 2, 206: 2, 114: 2, 441: 2, 52: 

In [0]:
# more training

In [261]:
sidx, eidx = 1900, 2000
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss_posdouble, batch_size=10, epochs=51)

   start index: 1900, end index: 2000
Loss at step 0: 0.24226
Loss at step 10: 0.23347
Loss at step 20: 0.21081
Loss at step 30: 0.23898
Loss at step 40: 0.19872
Loss at step 50: 0.19460


In [262]:
calc_loss_with_changing_negative_pair(0, 100, tripletloss_posdouble)
calc_loss_with_changing_negative_pair(900, 1000, tripletloss_posdouble)
calc_loss_with_changing_negative_pair(1900, 2000, tripletloss_posdouble)

tf.Tensor(0.082761444, shape=(), dtype=float32)
tf.Tensor(0.09500242, shape=(), dtype=float32)
tf.Tensor(0.1655381, shape=(), dtype=float32)


In [263]:
counter = calc_all_ranks_counter(test_extracted_features_df['doc_id'], complex_sim_dict)
print(counter)

Counter({1: 124, 2: 58, 3: 41, 6: 28, 4: 28, 5: 27, 8: 25, 7: 20, 18: 15, 14: 14, 9: 13, 10: 12, 13: 12, 21: 11, 12: 10, 17: 10, 35: 10, 42: 9, 11: 9, 15: 9, 40: 9, 19: 9, 28: 9, 27: 9, 53: 8, 32: 8, 20: 8, 49: 7, 29: 7, 41: 7, 37: 7, 16: 7, 25: 7, 43: 7, 26: 6, 50: 6, 58: 6, 24: 6, 73: 6, 47: 6, 142: 6, 34: 5, 75: 5, 22: 5, 23: 5, 135: 5, 33: 5, 111: 5, 92: 5, 107: 5, 38: 5, 44: 5, 110: 4, 60: 4, 138: 4, 74: 4, 57: 4, 126: 4, 62: 4, 103: 4, 31: 4, 56: 4, 91: 4, 175: 4, 221: 4, 67: 4, 127: 4, 72: 4, 61: 4, 100: 3, 30: 3, 39: 3, 106: 3, 183: 3, 55: 3, 81: 3, 125: 3, 166: 3, 120: 3, 117: 3, 84: 3, 136: 3, 102: 3, 69: 3, 139: 3, 59: 3, 70: 3, 63: 3, 104: 3, 325: 3, 361: 3, 485: 3, 370: 3, 130: 3, 98: 3, 66: 3, 77: 3, 244: 3, 425: 3, 95: 3, 105: 3, 205: 3, 82: 3, 508: 3, 71: 3, 113: 2, 64: 2, 1851: 2, 354: 2, 184: 2, 240: 2, 87: 2, 268: 2, 89: 2, 172: 2, 218: 2, 115: 2, 122: 2, 83: 2, 118: 2, 153: 2, 51: 2, 358: 2, 146: 2, 133: 2, 754: 2, 333: 2, 152: 2, 79: 2, 206: 2, 114: 2, 441: 2, 52: 

### Conclusion

Loss get lower and lower, but that does not imrpove final result.

# Investigate relation between loss and prediction score

Above trial suggest minimize loss does not lead to prediction improvements.
Is is because of overfitting? or our loss is so different from our objective?

To figure out these points, I investigate relationship between loss and final counter score in training set.


### Compare Identity model and normal model

Motivation: raw feature result is the best.
Is it because W and B is not properly learned? or loss is definitely lower than raw feature, but result is bad?

To compare raw feature to trained model, make output dimenstion the same and calculate IdentityModel loss and compare.

In [0]:
class IdentityModel(object):
    def frwrd_pass(self,X_train):
        return X_train

In [0]:
def train_one_difficulty(sidx, eidx, batch_size=10, epochs=41):
  print("   start index: {}, end index: {}".format(sidx,eidx))
  train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=batch_size, epochs=epochs)

def calc_loss_and_print_with_region(sidx, eidx):
  lossavg = calc_loss_with_changing_negative_pair_without_print(sidx, eidx)
  print(f"{sidx}:{eidx} {lossavg}")

def print_losses():
  calc_loss_and_print_with_region(0, 100)
  calc_loss_and_print_with_region(100, 200)
  calc_loss_and_print_with_region(900, 1000)
  calc_loss_and_print_with_region(1000, 1100)
  calc_loss_and_print_with_region(1900, 2000)

  
def model_to_counter(model):
  test_extracted_features_df = to_extracted_features(test_normalized_feature_dict_1000, model)
  grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, model)
  result_sim_dict = create_sim_dict(test_extracted_features_df, grants_extracted_features_df)
  return calc_all_ranks_counter(test_extracted_features_df['doc_id'], result_sim_dict)

In [0]:
model = IdentityModel()

In [35]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1900, 2000)

Instructions for updating:
Use tf.cast instead.
tf.Tensor(0.0033903401, shape=(), dtype=float32)
tf.Tensor(0.055143323, shape=(), dtype=float32)
tf.Tensor(0.11747384, shape=(), dtype=float32)


In [0]:
model = Model(input_shape=768, output_shape=768)

In [269]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.036743686, shape=(), dtype=float32)
tf.Tensor(0.12538862, shape=(), dtype=float32)
tf.Tensor(0.16120608, shape=(), dtype=float32)


In [270]:
%%time

for sidx, eidx in start_end_index_pairs:
    print("   start index: {}, end index: {}".format(sidx,eidx))
    train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=21)

   start index: 0, end index: 100
Loss at step 0: 0.04723
Loss at step 10: 0.02497
Loss at step 20: 0.01770
   start index: 900, end index: 1000
Loss at step 0: 0.11713
Loss at step 10: 0.09884
Loss at step 20: 0.07645
   start index: 1900, end index: 2000
Loss at step 0: 0.13400
Loss at step 10: 0.10765
Loss at step 20: 0.08786
CPU times: user 6min 37s, sys: 8.13 s, total: 6min 45s
Wall time: 5min 37s


In [271]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.0049074353, shape=(), dtype=float32)
tf.Tensor(0.026888654, shape=(), dtype=float32)
tf.Tensor(0.08084515, shape=(), dtype=float32)


For easy case, Identity model is better, overall, trained model seems a little better.

The definition of "easy" is based on raw feature, so this result might reasonable.

In [0]:
test_extracted_features_df = to_extracted_features(test_normalized_feature_dict_1000, model)
grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, model)
result_sim_dict = create_sim_dict(test_extracted_features_df, grants_extracted_features_df)

In [273]:
counter = calc_all_ranks_counter(test_extracted_features_df['doc_id'], result_sim_dict)
print(counter)

Counter({1: 183, 2: 83, 3: 51, 5: 37, 6: 27, 4: 23, 7: 21, 11: 16, 16: 16, 10: 15, 15: 14, 9: 13, 8: 13, 13: 12, 12: 12, 14: 12, 20: 10, 21: 10, 17: 10, 19: 8, 24: 8, 22: 8, 18: 7, 41: 7, 44: 7, 48: 7, 38: 7, 40: 7, 27: 7, 35: 7, 91: 6, 25: 6, 47: 6, 52: 6, 29: 6, 69: 6, 31: 6, 28: 6, 73: 5, 34: 5, 50: 5, 56: 5, 74: 5, 72: 5, 23: 5, 70: 5, 30: 5, 51: 5, 33: 4, 176: 4, 90: 4, 62: 4, 46: 4, 32: 4, 42: 4, 71: 4, 57: 4, 49: 4, 36: 4, 121: 4, 97: 4, 45: 4, 109: 4, 101: 3, 95: 3, 194: 3, 225: 3, 43: 3, 294: 3, 107: 3, 105: 3, 79: 3, 39: 3, 168: 3, 110: 3, 205: 3, 131: 3, 63: 3, 26: 3, 164: 3, 145: 3, 59: 3, 125: 3, 133: 3, 83: 3, 65: 3, 111: 2, 85: 2, 108: 2, 183: 2, 75: 2, 928: 2, 106: 2, 114: 2, 411: 2, 216: 2, 94: 2, 127: 2, 82: 2, 189: 2, 68: 2, 578: 2, 214: 2, 231: 2, 293: 2, 67: 2, 549: 2, 288: 2, 269: 2, 53: 2, 732: 2, 64: 2, 151: 2, 1314: 2, 103: 2, 879: 2, 122: 2, 346: 2, 224: 2, 219: 2, 126: 2, 499: 2, 159: 2, 77: 2, 160: 2, 254: 2, 366: 2, 342: 2, 76: 2, 61: 2, 270: 2, 96: 2, 206:

In [0]:
identity_model = IdentityModel()

In [0]:
identity_test_extracted_features_df = to_extracted_features(test_normalized_feature_dict_1000, identity_model)
identity_grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, identity_model)
identity_result_sim_dict = create_sim_dict(identity_test_extracted_features_df, identity_grants_extracted_features_df)

In [276]:
counter = calc_all_ranks_counter(identity_test_extracted_features_df['doc_id'], identity_result_sim_dict)
print(counter)

Counter({1: 178, 2: 73, 3: 46, 5: 40, 4: 38, 6: 31, 7: 19, 9: 17, 8: 16, 10: 15, 11: 13, 13: 13, 14: 13, 15: 12, 18: 12, 16: 11, 12: 10, 26: 9, 19: 9, 34: 9, 21: 8, 23: 8, 38: 7, 28: 7, 50: 6, 53: 6, 33: 6, 39: 6, 37: 6, 68: 5, 76: 5, 73: 5, 35: 5, 64: 5, 31: 5, 29: 5, 17: 5, 61: 5, 32: 5, 100: 5, 47: 5, 48: 4, 63: 4, 90: 4, 59: 4, 70: 4, 65: 4, 40: 4, 20: 4, 30: 4, 52: 4, 49: 4, 44: 4, 143: 4, 187: 3, 122: 3, 125: 3, 22: 3, 444: 3, 66: 3, 25: 3, 85: 3, 189: 3, 51: 3, 92: 3, 93: 3, 121: 3, 99: 3, 168: 3, 69: 3, 385: 3, 147: 3, 36: 3, 148: 3, 83: 3, 114: 3, 304: 3, 195: 3, 46: 3, 43: 3, 57: 3, 97: 2, 75: 2, 163: 2, 264: 2, 454: 2, 130: 2, 42: 2, 110: 2, 78: 2, 45: 2, 258: 2, 62: 2, 126: 2, 192: 2, 183: 2, 77: 2, 378: 2, 211: 2, 181: 2, 169: 2, 67: 2, 87: 2, 88: 2, 1182: 2, 155: 2, 89: 2, 112: 2, 617: 2, 411: 2, 173: 2, 209: 2, 485: 2, 615: 2, 105: 2, 213: 2, 396: 2, 24: 2, 227: 2, 179: 2, 696: 2, 133: 2, 517: 2, 174: 2, 72: 2, 413: 2, 505: 2, 571: 2, 128: 2, 262: 2, 157: 2, 415: 2, 2394

Getting better! The difference is output shape only.


Try more training.

First, I train for easy case much more. Compare to identity model,  our model may not learn enough for easy case.

In [277]:
sidx, eidx = 0, 100
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=61)

   start index: 0, end index: 100
Loss at step 0: 0.00079
Loss at step 10: 0.00213
Loss at step 20: 0.00443
Loss at step 30: 0.00106
Loss at step 40: 0.00863
Loss at step 50: 0.01044
Loss at step 60: 0.00000


In [278]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.0020887444, shape=(), dtype=float32)
tf.Tensor(0.024206998, shape=(), dtype=float32)
tf.Tensor(0.077805005, shape=(), dtype=float32)


In [281]:
counter = model_to_counter(model)
print(counter)

Counter({1: 177, 2: 85, 3: 46, 5: 34, 4: 32, 6: 28, 7: 22, 16: 17, 9: 15, 8: 14, 13: 14, 15: 13, 17: 13, 18: 13, 11: 12, 10: 12, 20: 10, 49: 10, 47: 10, 14: 10, 24: 9, 19: 8, 44: 7, 37: 7, 21: 7, 50: 7, 28: 7, 76: 7, 12: 6, 25: 6, 45: 6, 27: 6, 31: 5, 34: 5, 85: 5, 51: 5, 40: 5, 41: 5, 29: 5, 77: 5, 56: 5, 22: 5, 30: 5, 127: 4, 88: 4, 81: 4, 58: 4, 55: 4, 43: 4, 111: 4, 32: 4, 64: 4, 26: 4, 129: 4, 264: 4, 65: 4, 53: 4, 155: 4, 79: 4, 63: 4, 23: 4, 116: 4, 67: 3, 102: 3, 197: 3, 91: 3, 89: 3, 71: 3, 54: 3, 78: 3, 61: 3, 144: 3, 38: 3, 158: 3, 33: 3, 36: 3, 142: 3, 225: 3, 98: 3, 35: 3, 72: 3, 294: 3, 108: 3, 168: 3, 103: 3, 82: 3, 73: 2, 1053: 2, 139: 2, 220: 2, 223: 2, 74: 2, 75: 2, 57: 2, 490: 2, 141: 2, 132: 2, 384: 2, 453: 2, 113: 2, 148: 2, 106: 2, 93: 2, 313: 2, 154: 2, 69: 2, 1107: 2, 217: 2, 42: 2, 126: 2, 541: 2, 200: 2, 469: 2, 321: 2, 867: 2, 242: 2, 123: 2, 529: 2, 152: 2, 1857: 2, 143: 2, 273: 2, 426: 2, 192: 2, 170: 2, 235: 2, 262: 2, 62: 2, 118: 2, 48: 2, 249: 2, 1095: 2

Score improvement does not lead to result improvement.

In [282]:
sidx, eidx = 900, 1000
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=61)

   start index: 900, end index: 1000
Loss at step 0: 0.03686
Loss at step 10: 0.03265
Loss at step 20: 0.02348
Loss at step 30: 0.02980
Loss at step 40: 0.02287
Loss at step 50: 0.01434
Loss at step 60: 0.00117


In [283]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.003018151, shape=(), dtype=float32)
tf.Tensor(0.007677373, shape=(), dtype=float32)
tf.Tensor(0.037012834, shape=(), dtype=float32)


Still improving. Train more for this difficulty.

In [284]:
sidx, eidx = 900, 1000
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=61)

   start index: 900, end index: 1000
Loss at step 0: 0.01103
Loss at step 10: 0.00866
Loss at step 20: 0.01300
Loss at step 30: 0.02034
Loss at step 40: 0.01726
Loss at step 50: 0.00692
Loss at step 60: 0.00000


In [285]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.0030531285, shape=(), dtype=float32)
tf.Tensor(0.005217336, shape=(), dtype=float32)
tf.Tensor(0.02647508, shape=(), dtype=float32)


In [286]:
sidx, eidx = 1900, 2000
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=61)

   start index: 1900, end index: 2000
Loss at step 0: 0.02518
Loss at step 10: 0.01456
Loss at step 20: 0.01335
Loss at step 30: 0.00632
Loss at step 40: 0.00237
Loss at step 50: 0.01831
Loss at step 60: 0.03654


In [287]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.0043739523, shape=(), dtype=float32)
tf.Tensor(0.004369103, shape=(), dtype=float32)
tf.Tensor(0.01745218, shape=(), dtype=float32)


In [288]:
counter = model_to_counter(model)
print(counter)

Counter({1: 199, 2: 83, 3: 69, 4: 47, 5: 40, 6: 28, 11: 23, 10: 21, 9: 21, 12: 18, 7: 17, 8: 16, 15: 15, 13: 13, 17: 13, 14: 12, 21: 12, 20: 12, 16: 10, 57: 9, 46: 9, 22: 9, 24: 9, 18: 9, 44: 8, 25: 8, 29: 8, 51: 7, 37: 7, 38: 7, 36: 7, 45: 7, 40: 7, 30: 6, 26: 6, 31: 6, 33: 6, 50: 6, 47: 6, 69: 6, 27: 6, 42: 5, 41: 5, 19: 5, 64: 5, 32: 5, 28: 5, 73: 5, 34: 5, 92: 4, 72: 4, 61: 4, 65: 4, 35: 4, 85: 4, 94: 4, 23: 4, 103: 4, 66: 4, 76: 4, 109: 4, 241: 3, 124: 3, 87: 3, 75: 3, 60: 3, 96: 3, 59: 3, 52: 3, 249: 3, 39: 3, 152: 3, 77: 3, 68: 3, 49: 3, 203: 3, 285: 2, 132: 2, 332: 2, 167: 2, 81: 2, 90: 2, 127: 2, 221: 2, 89: 2, 201: 2, 100: 2, 102: 2, 86: 2, 777: 2, 368: 2, 58: 2, 157: 2, 137: 2, 111: 2, 292: 2, 236: 2, 418: 2, 115: 2, 281: 2, 311: 2, 54: 2, 114: 2, 108: 2, 80: 2, 283: 2, 144: 2, 104: 2, 208: 2, 131: 2, 110: 2, 116: 2, 145: 2, 91: 2, 176: 2, 139: 2, 424: 2, 199: 2, 119: 2, 56: 2, 358: 2, 165: 2, 126: 1, 93: 1, 71: 1, 1257: 1, 158: 1, 1525: 1, 434: 1, 207: 1, 327: 1, 691: 1, 44

In [289]:
sum(counter[i] for i in range(11))

541

raw feature is 473. improved, but not so impressive.

- Output dimension seems important (100 might be too small)
- Lowering loss seems somewhat related to improving score, but the relation is not so firm.

### Make model more complex with the same output and middle layer

Improving loss lead to final result improvement.
But the loss we need to achive seems much lower than we first expect.

Try more complex model with lower loss more than normal model.
If the real cause is loss is not enough low, this triay must improve final result.

In [0]:
model = ComplexModel(input_shape=768, output_shape1=768, output_shape2=768)

In [291]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(100, 200)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1000, 1100)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.067356534, shape=(), dtype=float32)
tf.Tensor(0.09547331, shape=(), dtype=float32)
tf.Tensor(0.13949102, shape=(), dtype=float32)
tf.Tensor(0.1426682, shape=(), dtype=float32)
tf.Tensor(0.16762954, shape=(), dtype=float32)


In [292]:
sidx, eidx = 0, 100
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=61)

   start index: 0, end index: 100
Loss at step 0: 0.07791
Loss at step 10: 0.02279
Loss at step 20: 0.01264
Loss at step 30: 0.00411
Loss at step 40: 0.00503
Loss at step 50: 0.01039
Loss at step 60: 0.00108


In [293]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(100, 200)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1000, 1100)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.0034555183, shape=(), dtype=float32)
tf.Tensor(0.009102334, shape=(), dtype=float32)
tf.Tensor(0.04942668, shape=(), dtype=float32)
tf.Tensor(0.05588448, shape=(), dtype=float32)
tf.Tensor(0.11338662, shape=(), dtype=float32)


In [294]:
sidx, eidx = 0, 100
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=21)

   start index: 0, end index: 100
Loss at step 0: 0.00321
Loss at step 10: 0.00411
Loss at step 20: 0.00277


In [295]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(100, 200)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1000, 1100)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.0022882402, shape=(), dtype=float32)
tf.Tensor(0.0061722556, shape=(), dtype=float32)
tf.Tensor(0.03827481, shape=(), dtype=float32)
tf.Tensor(0.044389553, shape=(), dtype=float32)
tf.Tensor(0.10048514, shape=(), dtype=float32)


In [296]:
sidx, eidx = 0, 100
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=21)

   start index: 0, end index: 100
Loss at step 0: 0.00005
Loss at step 10: 0.00290
Loss at step 20: 0.00135


In [297]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(100, 200)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1000, 1100)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.001631824, shape=(), dtype=float32)
tf.Tensor(0.0045466432, shape=(), dtype=float32)
tf.Tensor(0.031644385, shape=(), dtype=float32)
tf.Tensor(0.037194483, shape=(), dtype=float32)
tf.Tensor(0.090768576, shape=(), dtype=float32)


Still improving...

In [298]:
sidx, eidx = 0, 100
print("   start index: {}, end index: {}".format(sidx,eidx))
train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=41)

   start index: 0, end index: 100
Loss at step 0: 0.00000
Loss at step 10: 0.00143
Loss at step 20: 0.00005
Loss at step 30: 0.00000
Loss at step 40: 0.00000


In [299]:
calc_loss_with_changing_negative_pair(0, 100)
calc_loss_with_changing_negative_pair(100, 200)
calc_loss_with_changing_negative_pair(900, 1000)
calc_loss_with_changing_negative_pair(1000, 1100)
calc_loss_with_changing_negative_pair(1900, 2000)

tf.Tensor(0.0010590523, shape=(), dtype=float32)
tf.Tensor(0.0029666373, shape=(), dtype=float32)
tf.Tensor(0.024927925, shape=(), dtype=float32)
tf.Tensor(0.029643761, shape=(), dtype=float32)
tf.Tensor(0.079145215, shape=(), dtype=float32)


Seems enough. Check current result.

In [303]:
counter = model_to_counter(model)
print(counter)

Counter({1: 153, 2: 76, 3: 40, 4: 38, 5: 30, 6: 25, 7: 18, 9: 18, 8: 17, 11: 14, 19: 13, 10: 13, 20: 13, 12: 11, 17: 11, 23: 11, 14: 9, 18: 9, 32: 8, 26: 8, 16: 8, 33: 7, 22: 7, 45: 7, 38: 7, 25: 7, 15: 7, 28: 7, 44: 7, 47: 6, 40: 6, 110: 6, 27: 6, 104: 6, 178: 5, 24: 5, 73: 5, 21: 5, 39: 5, 34: 5, 56: 5, 13: 5, 52: 5, 59: 5, 72: 5, 30: 4, 167: 4, 130: 4, 95: 4, 185: 4, 79: 4, 70: 4, 54: 4, 35: 4, 49: 4, 37: 4, 42: 4, 121: 4, 506: 3, 132: 3, 108: 3, 98: 3, 75: 3, 58: 3, 128: 3, 88: 3, 103: 3, 51: 3, 64: 3, 203: 3, 83: 3, 236: 3, 50: 3, 46: 3, 63: 3, 43: 3, 190: 3, 82: 2, 134: 2, 1899: 2, 208: 2, 148: 2, 77: 2, 1156: 2, 136: 2, 466: 2, 174: 2, 112: 2, 120: 2, 297: 2, 477: 2, 91: 2, 159: 2, 309: 2, 346: 2, 292: 2, 81: 2, 117: 2, 62: 2, 92: 2, 109: 2, 71: 2, 147: 2, 264: 2, 205: 2, 36: 2, 531: 2, 304: 2, 1024: 2, 89: 2, 404: 2, 186: 2, 31: 2, 371: 2, 429: 2, 143: 2, 55: 2, 78: 2, 61: 2, 129: 2, 249: 2, 29: 2, 311: 2, 600: 2, 219: 2, 224: 2, 328: 2, 57: 2, 663: 2, 196: 2, 193: 2, 187: 2, 3

Very bad even though loss seems better than Identity.
My hypotheses seems wrong. But training for more difficult pair.

In [308]:
train_one_difficulty(900, 1000, epochs=61)

   start index: 900, end index: 1000
Loss at step 0: 0.04020
Loss at step 10: 0.02869
Loss at step 20: 0.01957
Loss at step 30: 0.03560
Loss at step 40: 0.01328
Loss at step 50: 0.01476
Loss at step 60: 0.00290


In [309]:
print_losses()

0:100 0.0018382936250418425
100:200 0.0019187016878277063
900:1000 0.005641844589263201
1000:1100 0.007981248199939728
1900:2000 0.02922942116856575


In [310]:
train_one_difficulty(900, 1000, epochs=41)

   start index: 900, end index: 1000
Loss at step 0: 0.01570
Loss at step 10: 0.01210
Loss at step 20: 0.01037
Loss at step 30: 0.02991
Loss at step 40: 0.00579


In [311]:
print_losses()

0:100 0.0014890616293996572
100:200 0.00156587571837008
900:1000 0.00420421501621604
1000:1100 0.006300562992691994
1900:2000 0.024204019457101822


In [312]:
train_one_difficulty(900, 1000, epochs=41)

   start index: 900, end index: 1000
Loss at step 0: 0.01037
Loss at step 10: 0.00708
Loss at step 20: 0.00823
Loss at step 30: 0.02490
Loss at step 40: 0.00209


In [313]:
print_losses()

0:100 0.0011327170068398118
100:200 0.0012258307542651892
900:1000 0.00335480272769928
1000:1100 0.005342297721654177
1900:2000 0.02151733636856079


In [314]:
train_one_difficulty(900, 1000, epochs=41)

   start index: 900, end index: 1000
Loss at step 0: 0.00713
Loss at step 10: 0.00270
Loss at step 20: 0.00598
Loss at step 30: 0.01931
Loss at step 40: 0.00000


In [315]:
print_losses()

0:100 0.0009142808266915381
100:200 0.0010113270254805684
900:1000 0.0028058458119630814
1000:1100 0.0046907574869692326
1900:2000 0.01965579204261303


Still improving. But I don't know any metrics for overfitting this case.
And loss is already low enough.
Check result.

In [316]:
counter = model_to_counter(model)
print(counter)

Counter({1: 178, 2: 79, 3: 53, 4: 41, 5: 39, 6: 24, 8: 23, 9: 21, 10: 20, 12: 18, 11: 17, 15: 15, 18: 15, 21: 15, 16: 14, 7: 14, 20: 12, 17: 12, 22: 9, 31: 9, 38: 9, 13: 8, 24: 8, 35: 8, 39: 8, 27: 8, 14: 7, 23: 7, 43: 7, 25: 7, 33: 7, 49: 6, 28: 6, 126: 6, 75: 6, 42: 5, 36: 5, 74: 5, 76: 5, 56: 5, 85: 5, 44: 5, 67: 5, 50: 5, 30: 5, 26: 5, 34: 5, 51: 5, 37: 4, 102: 4, 32: 4, 66: 4, 84: 4, 121: 4, 52: 4, 80: 4, 125: 4, 93: 4, 60: 4, 57: 4, 210: 4, 58: 3, 61: 3, 72: 3, 134: 3, 136: 3, 48: 3, 107: 3, 152: 3, 108: 3, 339: 3, 54: 3, 97: 3, 53: 3, 200: 3, 147: 3, 118: 3, 69: 3, 73: 3, 86: 3, 104: 3, 124: 3, 70: 3, 64: 3, 159: 3, 139: 3, 129: 2, 222: 2, 89: 2, 71: 2, 127: 2, 214: 2, 98: 2, 103: 2, 47: 2, 113: 2, 162: 2, 193: 2, 206: 2, 237: 2, 46: 2, 148: 2, 119: 2, 29: 2, 157: 2, 41: 2, 543: 2, 275: 2, 165: 2, 318: 2, 502: 2, 130: 2, 208: 2, 433: 2, 259: 2, 77: 2, 114: 2, 245: 2, 83: 2, 223: 2, 40: 2, 250: 2, 185: 2, 160: 2, 62: 2, 442: 2, 99: 2, 377: 2, 252: 2, 59: 2, 144: 2, 128: 2, 90: 2,

Score is bad.
But training loss seems low enough.

I want to invesitage whether this is because of

1. overfitting
2. loss criteria is not adeaquate

If 1, training score must be good.
So I want to predict to predict setand check the result.

In [0]:
def calc_train_counter(model):
  train_extracted_features_df = to_extracted_features(train_normalized_feature_dict_1000, model)
  grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, model)
  result_sim_dict = create_sim_dict(train_extracted_features_df, grants_extracted_features_df)
  return calc_all_ranks_counter(train_extracted_features_df['doc_id'], result_sim_dict)

In [0]:
  train_extracted_features_df = to_extracted_features(train_normalized_feature_dict_1000, model)
  grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, model)
  result_sim_dict = create_sim_dict(train_extracted_features_df, grants_extracted_features_df)
  counter = calc_all_ranks_counter(train_extracted_features_df['doc_id'], result_sim_dict)

In [319]:
print(counter)

Counter({1: 174, 2: 79, 3: 56, 6: 41, 4: 33, 5: 28, 9: 24, 14: 21, 21: 21, 8: 21, 16: 18, 11: 18, 13: 17, 12: 17, 7: 17, 10: 16, 22: 15, 18: 14, 17: 14, 26: 11, 34: 11, 15: 11, 54: 10, 27: 10, 25: 8, 30: 8, 20: 8, 19: 8, 29: 8, 23: 8, 41: 8, 32: 8, 37: 8, 35: 7, 43: 7, 33: 7, 38: 7, 50: 6, 74: 6, 73: 6, 40: 6, 24: 6, 42: 6, 36: 6, 99: 6, 81: 5, 28: 5, 119: 5, 45: 5, 86: 5, 98: 5, 39: 5, 92: 5, 44: 5, 31: 4, 76: 4, 178: 4, 51: 4, 47: 4, 63: 4, 134: 4, 89: 4, 70: 4, 53: 4, 148: 4, 141: 4, 93: 4, 64: 4, 55: 4, 133: 4, 68: 3, 48: 3, 71: 3, 67: 3, 116: 3, 122: 3, 106: 3, 240: 3, 62: 3, 174: 3, 110: 3, 90: 3, 152: 3, 88: 3, 52: 3, 56: 3, 59: 3, 57: 3, 75: 3, 49: 3, 203: 3, 58: 3, 103: 3, 277: 3, 66: 3, 69: 3, 139: 3, 125: 3, 263: 3, 159: 3, 182: 2, 143: 2, 282: 2, 78: 2, 135: 2, 213: 2, 320: 2, 77: 2, 216: 2, 269: 2, 105: 2, 114: 2, 206: 2, 453: 2, 96: 2, 123: 2, 181: 2, 94: 2, 83: 2, 541: 2, 238: 2, 353: 2, 207: 2, 115: 2, 227: 2, 97: 2, 136: 2, 241: 2, 252: 2, 375: 2, 183: 2, 46: 2, 146: 2

Not so impressive. Check IdentityModel result.

In [0]:
identity_model = IdentityModel()

In [0]:
  ident_train_extracted_features_df = to_extracted_features(train_normalized_feature_dict_1000, identity_model)
  ident_grants_extracted_features_df = to_extracted_features(grants_normalized_feature_dict_2000, identity_model)
  ident_result_sim_dict = create_sim_dict(ident_train_extracted_features_df, ident_grants_extracted_features_df)
  ident_counter = calc_all_ranks_counter(ident_train_extracted_features_df['doc_id'], ident_result_sim_dict)

In [322]:
print(ident_counter)

Counter({1: 173, 2: 83, 3: 42, 4: 34, 6: 30, 5: 29, 9: 27, 12: 19, 8: 18, 10: 15, 11: 14, 7: 14, 26: 12, 14: 12, 13: 12, 19: 12, 15: 12, 23: 12, 21: 10, 16: 10, 37: 10, 20: 9, 32: 9, 17: 9, 28: 8, 25: 8, 18: 8, 44: 7, 36: 7, 22: 7, 33: 7, 24: 7, 31: 6, 40: 6, 42: 6, 45: 6, 53: 6, 82: 5, 47: 5, 59: 5, 58: 5, 79: 5, 30: 5, 34: 5, 43: 5, 98: 5, 65: 5, 100: 5, 74: 5, 77: 5, 48: 4, 63: 4, 68: 4, 219: 4, 113: 4, 60: 4, 155: 4, 76: 4, 78: 4, 57: 4, 39: 4, 27: 4, 38: 3, 54: 3, 337: 3, 41: 3, 92: 3, 176: 3, 141: 3, 227: 3, 75: 3, 49: 3, 102: 3, 50: 3, 115: 3, 46: 3, 61: 3, 184: 3, 119: 3, 89: 3, 199: 2, 195: 2, 173: 2, 73: 2, 579: 2, 148: 2, 2161: 2, 270: 2, 215: 2, 368: 2, 265: 2, 158: 2, 153: 2, 658: 2, 163: 2, 180: 2, 117: 2, 272: 2, 243: 2, 106: 2, 144: 2, 356: 2, 321: 2, 85: 2, 124: 2, 132: 2, 704: 2, 101: 2, 29: 2, 52: 2, 953: 2, 135: 2, 369: 2, 224: 2, 231: 2, 758: 2, 453: 2, 91: 2, 64: 2, 70: 2, 120: 2, 283: 2, 374: 2, 200: 2, 1627: 2, 238: 2, 242: 2, 571: 2, 264: 2, 151: 2, 373: 2, 238

Almost the same result. And this result is almost the same as test set result (and a little different)
It reasonable.

**Conclusion: Lowering this loss does not lead to good result.**

But why?

Investigate loss of some specific example.

In [0]:
random.seed(0)
test = create_training_input_np(0, 100)

In [0]:
test_ten = tf.convert_to_tensor(test, dtype=tf.float32)
anchor_data, positive_data, negative_data = test_ten

In [0]:
anchor_out = model.frwrd_pass(anchor_data[0:10])
positive_out = model.frwrd_pass(positive_data[0:10])
negative_out = model.frwrd_pass(negative_data[0:10])
curr_loss = tripletloss(anchor_out, positive_out, negative_out)

In [334]:
curr_loss

<tf.Tensor: id=57329251, shape=(), dtype=float32, numpy=0.0>

In [0]:
norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)

In [0]:
d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)

In [337]:
d_pos

<tf.Tensor: id=57329287, shape=(10, 1), dtype=float32, numpy=
array([[0.6817989 ],
       [0.7230514 ],
       [0.5686636 ],
       [0.2676103 ],
       [0.48857576],
       [0.36110663],
       [0.474051  ],
       [0.44065487],
       [0.35019422],
       [0.39531386]], dtype=float32)>

In [338]:
d_neg

<tf.Tensor: id=57329301, shape=(10, 1), dtype=float32, numpy=
array([[1.2124332],
       [1.2268171],
       [1.20551  ],
       [1.1653724],
       [1.045686 ],
       [1.1525494],
       [1.2070824],
       [1.1135503],
       [0.9210738],
       [1.0298408]], dtype=float32)>

Negative is large, but positive is not so close.

Our final prediction is just find closest neighbor.
This is one reason that lowering loss does not mean similar one closer.

But how about more close negative case?


In [0]:
random.seed(0)
test = create_training_input_np(1900, 2000)

In [0]:
test_ten = tf.convert_to_tensor(test, dtype=tf.float32)
anchor_data, positive_data, negative_data = test_ten
anchor_out = model.frwrd_pass(anchor_data[0:10])
positive_out = model.frwrd_pass(positive_data[0:10])
negative_out = model.frwrd_pass(negative_data[0:10])
curr_loss = tripletloss(anchor_out, positive_out, negative_out)

In [342]:
curr_loss

<tf.Tensor: id=57329411, shape=(), dtype=float32, numpy=0.01797967>

In [0]:
norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)
d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)

In [344]:
d_pos

<tf.Tensor: id=57329447, shape=(10, 1), dtype=float32, numpy=
array([[0.6817989 ],
       [0.7230514 ],
       [0.5686636 ],
       [0.2676103 ],
       [0.48857576],
       [0.36110663],
       [0.474051  ],
       [0.44065487],
       [0.35019422],
       [0.39531386]], dtype=float32)>

In [345]:
d_neg

<tf.Tensor: id=57329461, shape=(10, 1), dtype=float32, numpy=
array([[0.7595509 ],
       [0.97673   ],
       [0.79981047],
       [0.9308967 ],
       [0.631027  ],
       [0.7019844 ],
       [0.69824123],
       [0.8044194 ],
       [0.82447743],
       [0.7774799 ]], dtype=float32)>

Seems even similar example, they just make negative one to away.

Try identity model.

In [0]:
anchor_out = identity_model.frwrd_pass(anchor_data[0:10])
positive_out = identity_model.frwrd_pass(positive_data[0:10])
negative_out = identity_model.frwrd_pass(negative_data[0:10])
curr_loss = tripletloss(anchor_out, positive_out, negative_out)

In [348]:
curr_loss

<tf.Tensor: id=57329531, shape=(), dtype=float32, numpy=0.095065005>

In [0]:
norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)
d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)

In [350]:
d_pos

<tf.Tensor: id=57329567, shape=(10, 1), dtype=float32, numpy=
array([[0.43431515],
       [0.2820537 ],
       [0.33015478],
       [0.12734157],
       [0.19363195],
       [0.12262952],
       [0.24435222],
       [0.18245894],
       [0.1227845 ],
       [0.13579476]], dtype=float32)>

In [351]:
d_neg

<tf.Tensor: id=57329581, shape=(10, 1), dtype=float32, numpy=
array([[0.42137587],
       [0.42301697],
       [0.42255068],
       [0.314467  ],
       [0.26410198],
       [0.27452755],
       [0.2767427 ],
       [0.27861154],
       [0.27544433],
       [0.2740283 ]], dtype=float32)>

Both are smaller. It's natural that d_neg is small because this is the definition of "difficult".

Hypotheses:

Negative sample seems too small?
More versatile negative sample cause model to decide to loate positive closer.

In [0]:
NEGATIVE_SAMPLE_RATIO=10

In [63]:
print_losses()

0:100 0.00155129365157336
100:200 0.0037491279654204845
900:1000 0.029180005192756653
1000:1100 0.03348637744784355
1900:2000 0.08770105242729187


In [0]:
NEGATIVE_SAMPLE_RATIO=1

In [65]:
print_losses()

0:100 0.0015728222206234932
100:200 0.0036995408590883017
900:1000 0.0294814296066761
1000:1100 0.034682419151067734
1900:2000 0.08755087107419968


No visible difference. So learning result will be also similar...

### Hypotheses: 1900:2000 might be not enough

grants have 2524. Top 524 is still not enough for our final result check (we don't care about top 500, but just top 10)

Note. Instance is restearted. So I re-train and current model is a little different from previous overfitted one.

In [77]:
print_losses()

0:100 0.0038403773214668036
100:200 0.0030226900707930326
900:1000 0.004455043934285641
1000:1100 0.006801951676607132
1900:2000 0.020214032381772995


In [0]:
random.seed(0)
test = create_training_input_np(2400, 2500)
test_ten = tf.convert_to_tensor(test, dtype=tf.float32)
anchor_data, positive_data, negative_data = test_ten

In [0]:
anchor_out = model.frwrd_pass(anchor_data[0:10])
positive_out = model.frwrd_pass(positive_data[0:10])
negative_out = model.frwrd_pass(negative_data[0:10])
curr_loss = tripletloss(anchor_out, positive_out, negative_out)

In [0]:
norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)
d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)

In [83]:
curr_loss

<tf.Tensor: id=13151883, shape=(), dtype=float32, numpy=0.071649976>

In [84]:
d_pos

<tf.Tensor: id=13151918, shape=(10, 1), dtype=float32, numpy=
array([[0.6538392 ],
       [0.519831  ],
       [0.51159793],
       [0.21408975],
       [0.45576078],
       [0.30274475],
       [0.38854128],
       [0.43333435],
       [0.314139  ],
       [0.3563041 ]], dtype=float32)>

In [85]:
d_neg

<tf.Tensor: id=13151932, shape=(10, 1), dtype=float32, numpy=
array([[0.5881821 ],
       [0.5688733 ],
       [0.6746192 ],
       [0.46505594],
       [0.5540833 ],
       [0.57176256],
       [0.536672  ],
       [0.5353457 ],
       [0.50276816],
       [0.6260102 ]], dtype=float32)>

In [89]:
print(calc_train_counter(model))

Counter({1: 181, 2: 74, 4: 48, 3: 44, 5: 36, 6: 30, 8: 28, 7: 25, 14: 22, 9: 19, 10: 19, 11: 16, 13: 15, 12: 15, 21: 15, 35: 14, 36: 14, 29: 13, 18: 13, 19: 12, 25: 12, 17: 11, 16: 11, 15: 10, 20: 10, 23: 10, 30: 9, 32: 9, 24: 9, 45: 8, 26: 8, 51: 8, 85: 8, 27: 7, 39: 7, 40: 6, 43: 6, 31: 6, 48: 6, 55: 6, 79: 6, 66: 6, 61: 6, 42: 6, 54: 5, 47: 5, 50: 5, 92: 5, 72: 5, 67: 5, 34: 5, 189: 5, 70: 5, 90: 5, 84: 5, 22: 5, 69: 5, 111: 4, 38: 4, 46: 4, 151: 4, 53: 4, 122: 4, 68: 4, 76: 4, 59: 4, 65: 4, 28: 4, 164: 4, 49: 4, 62: 4, 44: 4, 33: 4, 113: 4, 181: 4, 95: 4, 157: 4, 71: 4, 82: 4, 56: 3, 149: 3, 140: 3, 63: 3, 58: 3, 60: 3, 74: 3, 120: 3, 80: 3, 251: 3, 106: 3, 176: 3, 117: 3, 170: 3, 86: 3, 57: 3, 77: 3, 207: 2, 167: 2, 163: 2, 133: 2, 478: 2, 37: 2, 89: 2, 102: 2, 87: 2, 211: 2, 142: 2, 240: 2, 371: 2, 213: 2, 187: 2, 296: 2, 201: 2, 303: 2, 114: 2, 269: 2, 155: 2, 137: 2, 148: 2, 78: 2, 109: 2, 168: 2, 209: 2, 241: 2, 108: 2, 387: 2, 449: 2, 41: 2, 99: 2, 428: 2, 91: 2, 125: 2, 274:

In [90]:
train_one_difficulty(2400, 2500, epochs=41)

   start index: 2400, end index: 2500
Loss at step 0: 0.07692
Loss at step 10: 0.09115
Loss at step 20: 0.10704
Loss at step 30: 0.08312
Loss at step 40: 0.06091


In [91]:
print_losses()

0:100 0.005918825510889292
100:200 0.004289938602596521
900:1000 0.004750657361000776
1000:1100 0.0069107962772250175
1900:2000 0.015909604728221893


In [92]:
print(calc_train_counter(model))

Counter({1: 188, 2: 85, 4: 48, 3: 47, 6: 39, 7: 35, 9: 30, 5: 29, 8: 25, 11: 25, 15: 21, 12: 21, 10: 20, 14: 19, 23: 14, 32: 14, 13: 13, 20: 12, 36: 12, 21: 12, 35: 11, 18: 11, 17: 11, 22: 10, 41: 10, 33: 10, 29: 10, 38: 9, 16: 9, 25: 9, 19: 9, 28: 9, 24: 9, 54: 7, 65: 7, 46: 7, 88: 7, 81: 7, 40: 6, 51: 6, 27: 6, 69: 6, 39: 6, 60: 6, 67: 6, 59: 6, 44: 6, 37: 6, 42: 6, 45: 6, 43: 5, 34: 5, 26: 5, 61: 4, 80: 4, 68: 4, 64: 4, 77: 4, 86: 4, 123: 4, 31: 4, 71: 4, 48: 4, 55: 4, 63: 4, 74: 3, 30: 3, 192: 3, 75: 3, 182: 3, 101: 3, 150: 3, 62: 3, 119: 3, 70: 3, 134: 3, 115: 3, 224: 3, 137: 3, 47: 3, 146: 3, 173: 3, 110: 3, 248: 2, 102: 2, 106: 2, 171: 2, 79: 2, 117: 2, 91: 2, 112: 2, 328: 2, 161: 2, 56: 2, 92: 2, 78: 2, 52: 2, 407: 2, 95: 2, 166: 2, 187: 2, 94: 2, 274: 2, 181: 2, 144: 2, 100: 2, 149: 2, 84: 2, 133: 2, 131: 2, 236: 2, 120: 2, 90: 2, 142: 2, 758: 2, 210: 2, 156: 2, 255: 2, 204: 2, 167: 2, 66: 2, 151: 2, 93: 2, 128: 2, 188: 2, 186: 2, 196: 2, 108: 2, 164: 2, 105: 2, 451: 1, 197: 1

In [0]:
anchor_out = model.frwrd_pass(anchor_data[0:10])
positive_out = model.frwrd_pass(positive_data[0:10])
negative_out = model.frwrd_pass(negative_data[0:10])
curr_loss = tripletloss(anchor_out, positive_out, negative_out)

In [94]:
curr_loss

<tf.Tensor: id=15281957, shape=(), dtype=float32, numpy=0.052411757>

In [0]:
norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)
d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)

In [96]:
d_pos

<tf.Tensor: id=15281993, shape=(10, 1), dtype=float32, numpy=
array([[0.62759006],
       [0.54049766],
       [0.49991387],
       [0.21470726],
       [0.46231097],
       [0.34099472],
       [0.40030754],
       [0.47132695],
       [0.35210943],
       [0.3938731 ]], dtype=float32)>

In [97]:
d_neg

<tf.Tensor: id=15282007, shape=(10, 1), dtype=float32, numpy=
array([[0.6204622 ],
       [0.5929167 ],
       [0.7050966 ],
       [0.47890544],
       [0.5733918 ],
       [0.6464665 ],
       [0.59796584],
       [0.593179  ],
       [0.5710293 ],
       [0.7072733 ]], dtype=float32)>

Seems improve. Train more.

In [98]:
train_one_difficulty(2400, 2500, epochs=41)

   start index: 2400, end index: 2500
Loss at step 0: 0.06085
Loss at step 10: 0.07161
Loss at step 20: 0.09513
Loss at step 30: 0.07689
Loss at step 40: 0.05186


In [99]:
print_losses()

0:100 0.005246615968644619
100:200 0.0036949277855455875
900:1000 0.004244708921760321
1000:1100 0.006112753879278898
1900:2000 0.01355723850429058


In [100]:
print(calc_train_counter(model))

Counter({1: 197, 2: 86, 3: 57, 4: 53, 5: 41, 6: 37, 8: 34, 7: 30, 10: 27, 14: 23, 9: 21, 12: 21, 13: 21, 11: 16, 19: 15, 15: 15, 16: 13, 29: 13, 31: 12, 20: 12, 18: 12, 32: 12, 33: 11, 23: 11, 24: 10, 25: 10, 26: 9, 34: 9, 27: 9, 30: 9, 21: 8, 28: 8, 59: 8, 39: 8, 37: 7, 17: 7, 42: 7, 74: 7, 73: 7, 40: 7, 38: 6, 53: 6, 57: 6, 66: 6, 49: 5, 43: 5, 61: 5, 22: 5, 72: 5, 55: 5, 58: 5, 36: 4, 71: 4, 76: 4, 108: 4, 47: 4, 51: 4, 89: 4, 45: 4, 46: 4, 48: 4, 122: 4, 77: 3, 136: 3, 69: 3, 86: 3, 110: 3, 131: 3, 54: 3, 50: 3, 175: 3, 41: 3, 85: 3, 154: 3, 159: 3, 79: 3, 124: 3, 97: 3, 64: 3, 145: 3, 75: 3, 130: 3, 114: 3, 63: 3, 190: 2, 35: 2, 93: 2, 56: 2, 211: 2, 78: 2, 129: 2, 157: 2, 208: 2, 52: 2, 117: 2, 87: 2, 169: 2, 100: 2, 225: 2, 65: 2, 187: 2, 369: 2, 116: 2, 102: 2, 174: 2, 82: 2, 112: 2, 267: 2, 139: 2, 88: 2, 84: 2, 167: 2, 138: 2, 166: 2, 168: 2, 229: 2, 98: 2, 220: 2, 44: 2, 141: 2, 194: 2, 177: 2, 360: 1, 133: 1, 263: 1, 215: 1, 83: 1, 296: 1, 558: 1, 158: 1, 70: 1, 81: 1, 237:

In [0]:
anchor_out = model.frwrd_pass(anchor_data[0:10])
positive_out = model.frwrd_pass(positive_data[0:10])
negative_out = model.frwrd_pass(negative_data[0:10])
curr_loss = tripletloss(anchor_out, positive_out, negative_out)

In [102]:
curr_loss

<tf.Tensor: id=17411987, shape=(), dtype=float32, numpy=0.04574002>

In [0]:
norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)
d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)

In [104]:
d_pos

<tf.Tensor: id=17412023, shape=(10, 1), dtype=float32, numpy=
array([[0.62602144],
       [0.5699303 ],
       [0.507542  ],
       [0.21538752],
       [0.46437764],
       [0.3683672 ],
       [0.41965628],
       [0.4970587 ],
       [0.37970722],
       [0.42225665]], dtype=float32)>

In [105]:
d_neg

<tf.Tensor: id=17412037, shape=(10, 1), dtype=float32, numpy=
array([[0.6695741 ],
       [0.6285114 ],
       [0.75068164],
       [0.47684312],
       [0.56854683],
       [0.69446784],
       [0.6275538 ],
       [0.6333555 ],
       [0.62374866],
       [0.7545363 ]], dtype=float32)>

In [106]:
train_one_difficulty(2400, 2500, epochs=41)

   start index: 2400, end index: 2500
Loss at step 0: 0.05625
Loss at step 10: 0.06148
Loss at step 20: 0.08764
Loss at step 30: 0.07247
Loss at step 40: 0.04698


In [107]:
print_losses()

0:100 0.004458918236196041
100:200 0.0031588852871209383
900:1000 0.003741171909496188
1000:1100 0.00533168762922287
1900:2000 0.01194639690220356


In [108]:
print(calc_train_counter(model))

Counter({1: 203, 2: 99, 3: 59, 4: 54, 5: 46, 6: 37, 7: 36, 14: 29, 8: 27, 12: 25, 9: 24, 10: 23, 11: 22, 17: 21, 15: 15, 19: 14, 24: 13, 18: 13, 30: 12, 23: 12, 21: 11, 38: 10, 26: 10, 32: 10, 13: 10, 40: 9, 25: 9, 28: 9, 34: 9, 44: 8, 22: 8, 20: 8, 31: 8, 59: 8, 27: 7, 35: 7, 48: 7, 29: 7, 41: 6, 63: 6, 56: 6, 33: 6, 47: 5, 64: 5, 51: 5, 102: 5, 43: 5, 116: 5, 69: 5, 39: 5, 37: 5, 53: 5, 36: 4, 54: 4, 52: 4, 83: 4, 60: 4, 160: 4, 73: 4, 62: 4, 101: 4, 45: 4, 96: 3, 89: 3, 76: 3, 71: 3, 67: 3, 16: 3, 104: 3, 170: 3, 72: 3, 88: 3, 46: 3, 58: 3, 120: 3, 66: 3, 158: 3, 152: 2, 90: 2, 84: 2, 79: 2, 74: 2, 68: 2, 168: 2, 103: 2, 327: 2, 91: 2, 49: 2, 80: 2, 127: 2, 209: 2, 65: 2, 239: 2, 98: 2, 196: 2, 162: 2, 145: 2, 50: 2, 85: 2, 93: 2, 190: 2, 132: 2, 114: 2, 61: 2, 180: 2, 144: 2, 137: 2, 92: 2, 169: 2, 172: 2, 77: 2, 214: 2, 100: 2, 276: 1, 212: 1, 177: 1, 228: 1, 523: 1, 195: 1, 106: 1, 140: 1, 249: 1, 57: 1, 479: 1, 173: 1, 1026: 1, 363: 1, 336: 1, 123: 1, 151: 1, 164: 1, 233: 1, 379

In [0]:
anchor_out = model.frwrd_pass(anchor_data[0:10])
positive_out = model.frwrd_pass(positive_data[0:10])
negative_out = model.frwrd_pass(negative_data[0:10])
curr_loss = tripletloss(anchor_out, positive_out, negative_out)

In [110]:
curr_loss

<tf.Tensor: id=19542017, shape=(), dtype=float32, numpy=0.04020954>

In [0]:
norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)
d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)

In [112]:
d_pos

<tf.Tensor: id=19542053, shape=(10, 1), dtype=float32, numpy=
array([[0.6248831 ],
       [0.5883431 ],
       [0.51430523],
       [0.2169357 ],
       [0.462008  ],
       [0.3871534 ],
       [0.43364465],
       [0.51372707],
       [0.39972734],
       [0.4423523 ]], dtype=float32)>

In [113]:
d_neg

<tf.Tensor: id=19542067, shape=(10, 1), dtype=float32, numpy=
array([[0.7111734 ],
       [0.657715  ],
       [0.78852314],
       [0.47675598],
       [0.55901486],
       [0.7273743 ],
       [0.64629453],
       [0.65896255],
       [0.6637303 ],
       [0.7854823 ]], dtype=float32)>

good for training set. how about test set?

In [116]:
counter = model_to_counter(model)
print(counter)

Counter({1: 194, 2: 104, 3: 60, 4: 47, 5: 26, 11: 25, 7: 25, 8: 25, 6: 24, 10: 23, 9: 20, 14: 18, 13: 18, 17: 16, 12: 14, 25: 13, 22: 12, 28: 10, 21: 10, 37: 10, 16: 9, 24: 8, 19: 8, 41: 8, 15: 8, 31: 8, 23: 7, 34: 7, 53: 7, 39: 7, 33: 7, 65: 6, 20: 6, 62: 6, 48: 6, 32: 6, 74: 6, 27: 6, 51: 5, 38: 5, 70: 5, 29: 5, 57: 5, 50: 5, 42: 5, 52: 5, 69: 4, 40: 4, 138: 4, 136: 4, 47: 4, 36: 4, 18: 4, 85: 4, 26: 4, 118: 4, 35: 4, 56: 4, 78: 4, 96: 4, 68: 3, 99: 3, 63: 3, 61: 3, 59: 3, 60: 3, 106: 3, 436: 3, 133: 3, 128: 3, 142: 3, 79: 3, 175: 3, 44: 3, 94: 3, 92: 3, 43: 3, 103: 3, 30: 3, 82: 3, 77: 3, 132: 3, 73: 3, 49: 3, 114: 2, 220: 2, 438: 2, 193: 2, 379: 2, 217: 2, 154: 2, 219: 2, 104: 2, 121: 2, 214: 2, 156: 2, 169: 2, 66: 2, 319: 2, 161: 2, 55: 2, 321: 2, 119: 2, 287: 2, 322: 2, 326: 2, 86: 2, 126: 2, 98: 2, 560: 2, 375: 2, 110: 2, 171: 2, 168: 2, 146: 2, 45: 2, 75: 2, 368: 2, 153: 2, 197: 2, 276: 2, 185: 2, 382: 2, 361: 2, 113: 2, 255: 1, 117: 1, 464: 1, 638: 1, 135: 1, 1283: 1, 275: 1, 

In [117]:
sum(counter[i] for i in range(11))

548

Seems best model.

In [0]:
checkpoint_directory = "/tmp/training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_directory, "ckpt")

In [0]:
os.makedirs('trained_model/tripletloss_complex_2400_2500', exist_ok=True)

In [122]:
len(model.variables)

4

In [0]:
checkpoint = tf.train.Checkpoint(model_var_0=model.variables[0], model_var_1=model.variables[1], model_var_2=model.variables[2], model_var_3=model.variables[3])

In [124]:
checkpoint.save(file_prefix='trained_model/tripletloss_complex_2400_2500/ckpt')

'trained_model/tripletloss_complex_2400_2500/ckpt-1'

In [126]:
!gsutil cp trained_model/tripletloss_complex_2400_2500/* gs://karino2-uspatent/mlstudy-phys/tripletloss_complex_2400_2500/

Copying file://trained_model/tripletloss_complex_2400_2500/checkpoint [Content-Type=application/octet-stream]...
Copying file://trained_model/tripletloss_complex_2400_2500/ckpt-1.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://trained_model/tripletloss_complex_2400_2500/ckpt-1.index [Content-Type=application/octet-stream]...
- [3 files][  4.5 MiB/  4.5 MiB]                                                
Operation completed over 3 objects/4.5 MiB.                                      


### Misc adhock Investigation (little insight)

In [0]:
train_one_difficulty(900, 1000, epochs=41)

   start index: 900, end index: 1000
Loss at step 0: 0.03616
Loss at step 10: 0.02962
Loss at step 20: 0.01970
Loss at step 30: 0.03002
Loss at step 40: 0.02664


In [74]:
print_losses()

0:100 0.0021061752922832966
100:200 0.0018963681068271399
900:1000 0.0066237072460353374
1000:1100 0.009160052984952927
1900:2000 0.03698155656456947


In [75]:
train_one_difficulty(1900, 2000, epochs=41)

   start index: 1900, end index: 2000
Loss at step 0: 0.04450
Loss at step 10: 0.02616
Loss at step 20: 0.01589
Loss at step 30: 0.00497
Loss at step 40: 0.01434


In [76]:
print_losses()

0:100 0.0038403773214668036
100:200 0.0030226900707930326
900:1000 0.004455043934285641
1000:1100 0.006801951676607132
1900:2000 0.020214032381772995


In [0]:
create_training_input_np(0, 100).shape

(3, 1282, 768)

In [0]:
NEGATIVE_SAMPLE_RATIO=10

In [0]:
create_training_input_np(0, 100).shape

(3, 12820, 768)

In [0]:
NEGATIVE_SAMPLE_RATIO=1

In [0]:
model = ComplexModel(input_shape=768, output_shape1=768, output_shape2=768)

In [53]:
train_one_difficulty(0, 100, epochs=101)

   start index: 0, end index: 100
Loss at step 0: 0.04871
Loss at step 10: 0.01886
Loss at step 20: 0.01255
Loss at step 30: 0.00501
Loss at step 40: 0.01140
Loss at step 50: 0.01591
Loss at step 60: 0.00045
Loss at step 70: 0.00000
Loss at step 80: 0.00000
Loss at step 90: 0.00000
Loss at step 100: 0.00865


In [71]:
train_one_difficulty(900, 1000, epochs=41)

   start index: 900, end index: 1000


KeyboardInterrupt: ignored

In [70]:
grants_target_df.shape

(2524, 2)

By the way, why d_neg sometime greater than  1?
Isn't it less than 1?

Investigate it. (instance is restarted... I use normal model for investigation)

I see. cos becomes negative (till -1) So it is OK to becomes greater than 1.

In [0]:
random.seed(0)
test = create_training_input_np(0, 100)
test_ten = tf.convert_to_tensor(test, dtype=tf.float32)
anchor_data, positive_data, negative_data = test_ten

anchor_out = model.frwrd_pass(anchor_data[0:10])
positive_out = model.frwrd_pass(positive_data[0:10])
negative_out = model.frwrd_pass(negative_data[0:10])
curr_loss = tripletloss(anchor_out, positive_out, negative_out)

norm_a_out = tf.nn.l2_normalize(anchor_out, axis=1)
norm_p_out = tf.nn.l2_normalize(positive_out, axis=1)
norm_n_out = tf.nn.l2_normalize(negative_out, axis=1)
d_pos = tf.losses.cosine_distance(norm_a_out, norm_p_out, axis=1, reduction=tf.losses.Reduction.NONE)
d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)

In [43]:
d_neg

<tf.Tensor: id=2821385, shape=(10, 1), dtype=float32, numpy=
array([[0.66342866],
       [0.6722027 ],
       [0.66510904],
       [0.52627015],
       [0.50272846],
       [0.5830833 ],
       [0.5462886 ],
       [0.56473255],
       [0.4612081 ],
       [0.46651566]], dtype=float32)>

In [0]:
d_neg = tf.losses.cosine_distance(norm_a_out, norm_n_out, axis=1, reduction=tf.losses.Reduction.NONE)

In [45]:
norm_n_out.shape

TensorShape([Dimension(10), Dimension(768)])

In [46]:
np.linalg.norm(norm_n_out[0])

1.0

In [0]:
np.linalg.norm(norm_n_out[1])

0.9999999

In [47]:
1- sum(norm_a_out[0]*norm_n_out[0])

<tf.Tensor: id=2825257, shape=(), dtype=float32, numpy=0.6634288>

In [48]:
d_neg[0]

<tf.Tensor: id=2825262, shape=(1,), dtype=float32, numpy=array([0.66342866], dtype=float32)>

In [0]:
curr_loss

<tf.Tensor: id=57329251, shape=(), dtype=float32, numpy=0.0>

In [0]:
norm_n_out.shape

TensorShape([Dimension(10), Dimension(768)])

In [37]:
%%time

for sidx, eidx in start_end_index_pairs:
    print("   start index: {}, end index: {}".format(sidx,eidx))
    train_with_changing_negative_pair_with_loss(sidx, eidx, tripletloss, batch_size=10, epochs=21)

   start index: 0, end index: 100
Instructions for updating:
Colocations handled automatically by placer.
Loss at step 0: 0.05039
Loss at step 10: 0.02480
Loss at step 20: 0.01906
   start index: 900, end index: 1000
Loss at step 0: 0.11757
Loss at step 10: 0.09759
Loss at step 20: 0.07440
   start index: 1900, end index: 2000
Loss at step 0: 0.12522
Loss at step 10: 0.11053
Loss at step 20: 0.07883
CPU times: user 6min 15s, sys: 6.55 s, total: 6min 22s
Wall time: 5min 17s


In [39]:
print_losses()

0:100 0.00550646660849452
100:200 0.00757866445928812
900:1000 0.026651430875062943
1000:1100 0.030813390389084816
1900:2000 0.07903685420751572


In [0]:
random.seed(0)
trip = create_triplet_pairs(0, 100)

In [0]:
trip[0:5]

,app_id,cited_grants,uncited_grants
0,14222691,8206188,7474966
1,14222691,8177561,6925385
2,14222691,8179692,7735351
3,12515852,7235710,7271332
4,12033424,6950953,7371046


In [0]:
trip.shape

(1282, 3)

In [0]:
random.seed(0)
test = create_training_input_np(0, 100)

In [0]:
test.shape

(3, 1282, 768)

In [0]:
test_ten = tf.convert_to_tensor(test, dtype=tf.float32)
anchor_data, positive_data, negative_data = test_ten

In [0]:
anchor_data[0, 0:5]

<tf.Tensor: id=4237669, shape=(5,), dtype=float32, numpy=
array([-0.05969358,  0.05039638, -0.02005569,  0.0451927 , -0.01494433],
      dtype=float32)>

In [0]:
train_normalized_feature_dict_1000[14222691][0:5]

array([-0.05969358,  0.05039638, -0.0200557 ,  0.04519269, -0.01494433])

In [0]:
positive_data[0, 0:5]

<tf.Tensor: id=4237674, shape=(5,), dtype=float32, numpy=
array([-0.04778686,  0.03372966, -0.01699614,  0.0232354 ,  0.00648623],
      dtype=float32)>

In [0]:
grants_normalized_feature_dict_2000[8206188][0:5]

array([-0.04778686,  0.03372966, -0.01699614,  0.0232354 ,  0.00648623])

In [0]:
negative_data[0, 0:5]

<tf.Tensor: id=4237679, shape=(5,), dtype=float32, numpy=
array([-0.01261457,  0.01773252, -0.03752513,  0.02603934, -0.01327848],
      dtype=float32)>

In [0]:
grants_normalized_feature_dict_2000[7474966][0:5]

array([-0.01261457,  0.01773252, -0.03752513,  0.02603934, -0.01327848])

In [0]:
data_num = int(test.shape[1])

In [0]:
random.seed(0)
rand_idx = np.random.permutation(data_num)

In [0]:
rand_idx[0:5]

array([1102,  653,  475,   81,  404])

In [0]:
test2 = np.array([
    test[0][rand_idx], 
    test[1][rand_idx], 
    test[2][rand_idx]])

In [0]:
trip[1102:1103]

,app_id,cited_grants,uncited_grants
1102,12450212,7719957,7506657


In [0]:
test_ten2 = tf.convert_to_tensor(test2, dtype=tf.float32)
anchor_data2, positive_data2, negative_data2 = test_ten2

In [0]:
anchor_data2[0, 0:5]

<tf.Tensor: id=4237697, shape=(5,), dtype=float32, numpy=
array([-0.03815173,  0.04558022, -0.03306705,  0.00874347, -0.01623795],
      dtype=float32)>

In [0]:
train_normalized_feature_dict_1000[12450212][0:5]

array([-0.03815173,  0.04558022, -0.03306705,  0.00874347, -0.01623795])

In [0]:
positive_data2[0, 0:5]

<tf.Tensor: id=4237702, shape=(5,), dtype=float32, numpy=
array([-0.03455767,  0.03205481, -0.05553311, -0.00180341, -0.02068364],
      dtype=float32)>

In [0]:
grants_normalized_feature_dict_2000[7719957][0:5]

array([-0.03455768,  0.03205481, -0.05553311, -0.00180341, -0.02068364])

In [0]:
grants_normalized_feature_dict_2000[7506657][0:5]

array([-0.03321561,  0.02758702,  0.02755731,  0.05383265,  0.01525151])

In [0]:
negative_data2[0, 0:5]

<tf.Tensor: id=4237707, shape=(5,), dtype=float32, numpy=
array([-0.03321561,  0.02758702,  0.02755731,  0.05383265,  0.01525151],
      dtype=float32)>

In [0]:
anchor_out2 = model.frwrd_pass(anchor_data2[0:5])
positive_out2 = model.frwrd_pass(positive_data2[0:5])
negative_out2 = model.frwrd_pass(negative_data2[0:5])
curr_loss2 = tripletloss(anchor_out2, positive_out2, negative_out2)

In [324]:
curr_loss2

<tf.Tensor: id=57329091, shape=(), dtype=float32, numpy=0.0>

In [0]:
curr_loss2

<tf.Tensor: id=6980813, shape=(), dtype=float32, numpy=0.0>

In [0]:
norm_a_out2 = tf.nn.l2_normalize(anchor_out2, axis=1)
norm_p_out2 = tf.nn.l2_normalize(positive_out2, axis=1)
norm_n_out2 = tf.nn.l2_normalize(negative_out2, axis=1)

In [0]:
d_pos2 = tf.losses.cosine_distance(norm_a_out2, norm_p_out2, axis=1, reduction=tf.losses.Reduction.NONE)
d_neg2 = tf.losses.cosine_distance(norm_a_out2, norm_n_out2, axis=1, reduction=tf.losses.Reduction.NONE)

In [327]:
d_pos2

<tf.Tensor: id=57329127, shape=(5, 1), dtype=float32, numpy=
array([[0.5671426 ],
       [0.48774767],
       [0.3983801 ],
       [0.3131017 ],
       [0.31788284]], dtype=float32)>

In [328]:
d_neg2

<tf.Tensor: id=57329141, shape=(5, 1), dtype=float32, numpy=
array([[1.2046154 ],
       [0.93544954],
       [1.146873  ],
       [1.121481  ],
       [0.80375075]], dtype=float32)>

In [0]:
d_pos2

<tf.Tensor: id=6980849, shape=(5, 1), dtype=float32, numpy=
array([[0.32095897],
       [0.3099807 ],
       [0.24314821],
       [0.13694352],
       [0.18507373]], dtype=float32)>

In [0]:
d_neg2

<tf.Tensor: id=6980863, shape=(5, 1), dtype=float32, numpy=
array([[0.63673747],
       [0.5831015 ],
       [0.5028086 ],
       [0.7062503 ],
       [0.48968685]], dtype=float32)>

In [0]:
test_app_df.head()

,app_id,xml
0,14307191,"<us-patent-application lang=""EN"" dtd-version=""..."
1,13137006,"<us-patent-application lang=""EN"" dtd-version=""..."
2,12741959,"<us-patent-application lang=""EN"" dtd-version=""..."
3,12643447,"<us-patent-application lang=""EN"" dtd-version=""..."
4,14200253,"<us-patent-application lang=""EN"" dtd-version=""..."


In [0]:
list(reversed(sort_similarity_by_value(sim_dict_by_extracted, 14307191)))

[(7675444, 0.81486243),
 (8213360, 0.8095555),
 (8295300, 0.7975527),
 (7440728, 0.7911943),
 (7181534, 0.79116535),
 (7015791, 0.78744763),
 (7047561, 0.7843642),
 (7496099, 0.7797317),
 (6856519, 0.7784024),
 (7400687, 0.77812225),
 (7596647, 0.7777683),
 (6943925, 0.77549464),
 (6968158, 0.770937),
 (7430610, 0.77051735),
 (8149834, 0.7702616),
 (7197008, 0.76929677),
 (7080160, 0.76841784),
 (6927664, 0.768098),
 (7787480, 0.7665925),
 (7460369, 0.76627505),
 (7774833, 0.76552117),
 (6947376, 0.76504576),
 (6943668, 0.76469994),
 (7684209, 0.75894296),
 (7898105, 0.7584627),
 (7359643, 0.75701934),
 (7184876, 0.7568906),
 (7215144, 0.7568878),
 (7733589, 0.7557938),
 (7388946, 0.75565314),
 (7484979, 0.75525403),
 (7418208, 0.7550416),
 (7019617, 0.7520639),
 (6895220, 0.7517824),
 (7408379, 0.75157136),
 (7050761, 0.75155663),
 (7715309, 0.75128585),
 (7949309, 0.7512685),
 (8126094, 0.75113165),
 (8051342, 0.7495631),
 (7215775, 0.74950296),
 (7843906, 0.74846643),
 (7145862, 0.7

In [0]:
get_cited_grants(citations_info_target, 14307191)

{7576688}

In [0]:
sim_dict_by_extracted[14307191][7576688]

0.5689387

In [0]:
grants_extracted_features_df['parsed'].head()

0    6837383
1    6837647
2    6837799
3    6837893
4    6837910
Name: parsed, dtype: int64

In [0]:
grants_extracted_features_df[grants_extracted_features_df['parsed']==7576688].index[0]

1533

In [0]:
norm_p_out3 = grants_extracted_features_df['extracted_feature'][1533]

In [0]:
sum(norm_p_out3.shape)

100

In [0]:
test_extracted_features_df[test_extracted_features_df['app_id'] == 14307191].index[0]

890

In [0]:
norm_a_out3 = test_extracted_features_df['extracted_feature'][890]
norm_a_out3.shape

(100,)

In [0]:
tf.losses.cosine_distance(norm_a_out3, norm_p_out3, axis=0) 

<tf.Tensor: id=9827739, shape=(), dtype=float32, numpy=0.4310614>

In [0]:
0.5689387+tf.losses.cosine_distance(norm_a_out3, norm_p_out3, axis=0)

<tf.Tensor: id=9827768, shape=(), dtype=float32, numpy=1.0>

In [0]:
grants_extracted_features[0].sum()

7.989149